In [10]:
import numpy as np
import pandas as pd
import polars as pl
import sys
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px


pd.set_option('display.max_columns',None)
import psycopg2


#to scale the data using z-score 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Algorithms to use
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

#Metrics to evaluate the model
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve

import warnings
warnings.filterwarnings("ignore")

#importing PCA and TSNE
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [70]:
import pandas as pd
x = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/condition_specific/Condition_specific_all_lit_genes/Nephro_Lit_final_Positions_hg38_hg37.xlsx')
x['chrom'] = x['Chrom-pos-Ref-Alt_38'].str.split(',')
x = x.explode('chrom')

x['CHROM'] = x['chrom'].str.split('-').str[0]

# Function to add 'chr' prefix conditionally
def add_chr_prefix(chrom):
    if pd.notnull(chrom) and chrom.strip() != '':
        return 'chr' + str(chrom)
    else:
        return chrom

# Applying the function to the 'chromosome' column
x['CHROM'] = x['CHROM'].apply(add_chr_prefix)
x['CHROM'] = x['CHROM'].str.strip()
x['CHROM'] = x['CHROM'].str.replace(r'\s+', '')
x['POS'] = x['chrom'].str.split('-').str[1]
x['Ref'] = x['chrom'].str.split('-').str[2]
x['Alt'] = x['chrom'].str.split('-').str[3]


x.dropna(subset=['CHROM'], inplace=True)
# Drop rows with empty cells after removing leading and trailing whitespaces
x['CHROM'] = x['CHROM'].str.strip()
x['POS'] = x['POS'].str.strip()
# Dropping rows with empty cells and NaN values in both 'chromosome' and 'position' columns
x.dropna(subset=['Gene', 'CHROM', 'POS'], inplace=True)
df_3 = x[['Gene', 'CHROM', 'POS', 'Ref', 'Alt']]
df_3['Literature'] = 'Nephro' #################################################### Change the condition #################
df_3.drop_duplicates(subset='POS', inplace=True)
df_3['POS'] = df_3['POS'].astype('int64')
df_3 = df_3.reset_index()
df_3 = df_3[['Gene', 'CHROM', 'POS', 'Ref', 'Alt', 'Literature']]

df = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/KAPA HyperExome_hg38_capture_targets (1).bed', sep = '\t', header = None)
df.columns = ['chromosome', 'Start_pos', 'End_pos', 'INFO']

df['Extended_Start_pos'] = df['Start_pos'] - 20
df['Extended_End_pos'] = df['End_pos'] + 20

df['gene_symbol'] = df['INFO'].str.extract(r'gene_symbol=([^;]+)')
df = df[['chromosome', 'Extended_Start_pos', 'Extended_End_pos', 'INFO', 'gene_symbol']]


# Step 1: Create a dictionary from the df DataFrame
chromosome_dict = {}
for _, row in df.iterrows():
    chromosome = row['chromosome']
    start_pos = row['Extended_Start_pos']
    end_pos = row['Extended_End_pos']
    if chromosome not in chromosome_dict:
        chromosome_dict[chromosome] = []
    chromosome_dict[chromosome].append((start_pos, end_pos))

# Step 2: Define a function to check coverage
def check_coverage(row):
    pos = row['POS']
    chromosome = row['CHROM']
    if chromosome in chromosome_dict:
        ranges = chromosome_dict[chromosome]
        for start, end in ranges:
            if start <= pos <= end:
                return 'Covered'
    return 'Not_Covered'

# Step 3: Apply the function to create the new column in dataset2
df_3['Covered/Not_Covered'] = df_3.apply(check_coverage, axis=1)

df_3 = df_3[df_3['Covered/Not_Covered'] == 'Covered']
df_3 = df_3[['Gene', 'CHROM', 'POS', 'Ref', 'Alt', 'Literature']]
df_3.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/condition_specific/Condition_specific_all_lit_genes/Final_lit/Nephro_lit_variants.xlsx', index=False)
df_3

,Gene,CHROM,POS,Ref,Alt,Literature
0,ACE,chr17,63477072,AGCCGAGCACC,A,Nephro
1,ACE,chr17,63477106,CTCGGGCCGCC,C,Nephro
2,ACE,chr17,63477081,CCGCGCACCGC,C,Nephro
3,ACE,chr17,63477553,TGGCTGCTCCC,T,Nephro
5,ACE,chr17,63484349,TCCTCCAGGCC,T,Nephro
...,...,...,...,...,...,...
1737,WT1,chr11,32435148,C,A,Nephro
1738,WT1,chr11,32427940,C,T,Nephro
1741,WT1,chr11,32396399,T,G,Nephro
1742,WT1,chr11,32392787,G,T,Nephro


In [38]:
df = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/condition_specific/Condition_specific_all_lit_genes/Pancreatic_Cancer_Lit_final_Positions_hg38_hg37.csv', sep = '\t')
df.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/condition_specific/Condition_specific_all_lit_genes/Pancreatic_Cancer_Lit_final_Positions_hg38_hg37.xlsx', index=False)

In [2]:
import pandas as pd
df_genes = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/condition_specific/Condition_specific_all_lit_genes/Final_lit/21_Conditions_Genes_info.xlsx')
df_genes

,Gene,Condition,Headings,21_conditions_list
0,CTLA4,Type 1 Diabetes,Predisposition for diabetes,Diabetes
1,INS,Type 1 Diabetes,Predisposition for diabetes,Diabetes
2,IL2RA,Type 1 Diabetes,Predisposition for diabetes,Diabetes
3,HNF1A,Type 1 Diabetes,Predisposition for diabetes,Diabetes
4,CEL,Type 1 Diabetes,Predisposition for diabetes,Diabetes
...,...,...,...,...
757,RYR1,Obesity,Obesity,Obesity
758,SIM1,Obesity,Obesity,Obesity
759,BRCA2,Obesity,Obesity,Obesity
760,SCN1A,Obesity,Obesity,Obesity


In [3]:
result = df_genes.groupby('Gene')['Condition', 'Headings', '21_conditions_list'].agg(lambda x: '; '.join(x.unique())).reset_index()
result

,Gene,Condition,Headings,21_conditions_list
0,A2ML1,Other Skin Conditions,Skin Health,Skin Health
1,AAGAB,Dry Skin/fragile skin,Skin Health,Skin Health
2,ABCA1,Coronary artery disease risk; Polygenic (Famil...,Cardiac; Cholesterol disorders; Gallstones,"Cardiac Health(CAD, Arrythmias, Cardiomyopathy..."
3,ABCA12,Dry Skin/fragile skin,Skin Health,Skin Health
4,ABCB4,"Polygenic (Familial combined, small dense LDL,...",Cholesterol disorders,Cholesterol Disorders
...,...,...,...,...
579,XPC,Dry Skin/fragile skin,Skin Health,Skin Health
580,ZMPSTE24,"Polygenic (Familial combined, small dense LDL,...",Cholesterol disorders,Cholesterol Disorders
581,ZNF804A,Schizophrenia,Mood Disorder,"Mood Disorders (Anxiety, Schizophrenia, Depres..."
582,ZNRF1,Diabetic Retinopathy,Diabetic Retinopathy,Diabetes


In [5]:
result.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/condition_specific/Condition_specific_all_lit_genes/Final_lit/last_conditions/genes/All_condition_genes.xlsx', index=False)

In [6]:
df_1 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/condition_specific/Condition_specific_all_lit_genes/Final_lit/last_conditions/genes/All_condition_genes.xlsx')
df_1

,Gene,Condition,Headings,21_conditions_list
0,A2ML1,Other Skin Conditions,Skin Health,Skin Health
1,AAGAB,Dry Skin/fragile skin,Skin Health,Skin Health
2,ABCA1,Coronary artery disease risk; Polygenic (Famil...,Cardiac; Cholesterol disorders; Gallstones,"Cardiac Health(CAD, Arrythmias, Cardiomyopathy..."
3,ABCA12,Dry Skin/fragile skin,Skin Health,Skin Health
4,ABCB4,"Polygenic (Familial combined, small dense LDL,...",Cholesterol disorders,Cholesterol Disorders
...,...,...,...,...
579,XPC,Dry Skin/fragile skin,Skin Health,Skin Health
580,ZMPSTE24,"Polygenic (Familial combined, small dense LDL,...",Cholesterol disorders,Cholesterol Disorders
581,ZNF804A,Schizophrenia,Mood Disorder,"Mood Disorders (Anxiety, Schizophrenia, Depres..."
582,ZNRF1,Diabetic Retinopathy,Diabetic Retinopathy,Diabetes


In [20]:
df_2 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/condition_specific/Condition_specific_all_lit_genes/Final_lit/last_conditions/genes/genes_scores.xlsx')
df_2

,Gene Name,Gene_Score
0,TRDN-AS1,6
1,PMS2,8
2,AASS,4
3,ABCA13,4
4,ABCA7,8
...,...,...
3935,MLYCD,6
3936,ZSWIM6,4
3937,PDHX,6
3938,LOC105378457,4


In [8]:
merged_gene = pd.merge(df_1, df_2, on = 'Gene Name', how = 'left', sort=False)
merged_gene

NameError: name 'df_2' is not defined

In [7]:
merged_gene.merged_gene.isnull().sum()

NameError: name 'merged_gene' is not defined

In [ ]:
##################################### IMPORTING THE REQUIRED LIBRARIES ################################################

import numpy as np
import pandas as pd
import polars as pl
import sys
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px


pd.set_option('display.max_columns',None)
import psycopg2


#to scale the data using z-score 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Algorithms to use
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

#Metrics to evaluate the model
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve

import warnings
warnings.filterwarnings("ignore")

#importing PCA and TSNE
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

print("Loading the Librabies completed")

######################################### GETTING THE COVERED POSITIONS FROM THE MOTHER VCF FILE FROM THE BED COORDINATES ####

def read_bed_file(bed_file):
    bed_positions = set()
    with open(bed_file, 'r') as f:
        for line in f:
            if line.startswith('#'):  # Skip header lines if present
                continue
            fields = line.strip().split('\t')
            if len(fields) >= 3:
                chrom = fields[0]
                try:
                    start = int(fields[1])
                    end = int(fields[2])
                except ValueError:
                    continue  # Skip this line if start or end position is not an integer
                for pos in range(start, end + 1):
                    bed_positions.add((chrom, pos))
    return bed_positions

def normalize_chrom_name(chrom):
    return chrom.split('_')[0]

def filter_vcf_file(vcf_file, bed_positions):
    filtered_vcf_records = []
    with open(vcf_file, 'r') as f:
        for line in f:
            if line.startswith('#'):  # Preserve header lines in the output
                filtered_vcf_records.append(line)
                continue
            fields = line.strip().split('\t')
            if len(fields) >= 2:
                raw_chrom = fields[0]
                chrom = normalize_chrom_name(raw_chrom)
                try:
                    pos = int(fields[1])
                except ValueError:
                    continue  # Skip this line if 'POS' is not an integer
                if (chrom, pos) in bed_positions:
                    filtered_vcf_records.append(line)
    return filtered_vcf_records

def write_filtered_vcf(filtered_vcf_records, output_file):
    with open(output_file, 'w') as f:
        for record in filtered_vcf_records:
            f.write(record)

def main():
    bed_file = r'C:/Users/GenepoweRx_Madhu/Desktop/Covered_regions.bed'
    vcf_file = r'C:/Users/GenepoweRx_Madhu/Downloads/vcf_files_all/KHGLBS446_final109.vcf'
    output_file = r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/KHGLBS446_final109.vcf'

    bed_positions = read_bed_file(bed_file)
    filtered_vcf_records = filter_vcf_file(vcf_file, bed_positions)
    write_filtered_vcf(filtered_vcf_records, output_file)

if __name__ == "__main__":
    main()

print("Covered/Not_Covered completed")

    
########################################### IMPORTING THE VCF DATA AND EXPANDING THE DEPTH COLUMNS ##########################


vcf = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/COVERED_VCF_FILES_BED/KHGLBS446_final109.vcf', comment= '#', sep = '\t', header=None, low_memory=False)
vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']

sample_cols = vcf['SAMPLE'].str.split(':', expand=True)
sample_cols.columns = ['GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RBQ', 'ABQ', 'RDF', 'RDR', 'ADF', 'ADR']

# Assign the values to the newly created columns
vcf = pd.concat([vcf, sample_cols], axis=1)
vcf = vcf[['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL','RDF', 'RDR', 'ADF', 'ADR']]

print('Data Loading completed')

######################################### EXTRACTING THE ZYGOSITY FROM THE INFO COLUMN OF THE EACH VARIANT ##############

vcf['HET'] = vcf['INFO'].str.extract(r'HET=(\d)')
vcf['HOM'] = vcf['INFO'].str.extract(r'HOM=(\d)')

# Create a new column 'Zygosity' based on conditions
vcf['Zygosity'] = ''

vcf.loc[vcf['HOM'] == '1', 'Zygosity'] = 'Homozygous'
vcf.loc[vcf['HET'] == '1', 'Zygosity'] = 'Heterozygous'
vcf['GT'] = vcf['GT'].astype(str)

print('Zygosity extraction completed')

######################################## EXTRACTING THE GENEINFO FROM THE INFO COLUMN ####################################

vcf["Gene_Name"] = vcf["INFO"].str.extract('GENEINFO=(?P<GENEINFO>.+?);')
vcf['Gene Name'] = vcf['Gene_Name'].apply(lambda x: ','.join([segment.split(':')[0] for segment in x.split('|')]) if pd.notnull(x) else '')

print('Gene extraction completed')

####################################### SPLITTING AND EXPLODING THE CSQ COLUMN FOR THE REQUIRED PARAMETERS ##############

vcf['CSQ'] = vcf['INFO'].str.extract(r'CSQ=(.*)')
vcf['csq'] = vcf['CSQ'].str.split(',')
vcf = vcf.explode('csq')

print('CSQ splitting completed')

###################################### EXTRACTION OF THE REQUIRED KEY-VALUE PAIRS FROM THE CSQ ##########################

########################################################### Required columns extraction from the CSQ column ####################
vcf['ClinVar_CLNDN'] = vcf['csq'].str.split('|').str[82]
vcf['CLIN_SIG'] = vcf['csq'].str.split('|').str[70]
vcf['ClinVar_CLNREVSTAT'] = vcf['csq'].str.split('|').str[81]
vcf['ClinVar'] = vcf['csq'].str.split('|').str[79]
vcf['HGVSC'] = vcf['csq'].str.split('|').str[10]
vcf['HGVSP'] = vcf['csq'].str.split('|').str[11]
vcf['PolyPhen'] = vcf['csq'].str.split('|').str[38]
vcf['BIOTYPE'] = vcf['csq'].str.split('|').str[7]
vcf['EXON'] = vcf['csq'].str.split('|').str[8]
vcf['INTRON'] = vcf['csq'].str.split('|').str[9]
vcf['Protein_position'] = vcf['csq'].str.split('|').str[14]
vcf['Amino_acids'] = vcf['csq'].str.split('|').str[15]
vcf['Codons'] = vcf['csq'].str.split('|').str[16]
vcf['STRAND'] = vcf['csq'].str.split('|').str[19]
vcf['PUBMED'] = vcf['csq'].str.split('|').str[73]
vcf['Consequence'] = vcf['csq'].str.split('|').str[1]
vcf['IMPACT'] = vcf['csq'].str.split('|').str[2]
vcf['SIFT'] = vcf['csq'].str.split('|').str[37]
################################################## Frequency columns extraction ################################################
vcf['AF'] = vcf['csq'].str.split('|').str[42]
vcf['AFR_AF'] = vcf['csq'].str.split('|').str[43]
vcf['AMR_AF'] = vcf['csq'].str.split('|').str[44]
vcf['EAS_AF'] = vcf['csq'].str.split('|').str[45]
vcf['EUR_AF'] = vcf['csq'].str.split('|').str[46]
vcf['SAS_AF'] = vcf['csq'].str.split('|').str[47]
vcf['gnomADe_AF'] = vcf['csq'].str.split('|').str[48]
vcf['gnomADe_AFR_AF'] = vcf['csq'].str.split('|').str[49]
vcf['gnomADe_AMR_AF'] = vcf['csq'].str.split('|').str[50]
vcf['gnomADe_ASJ_AF'] = vcf['csq'].str.split('|').str[51]
vcf['gnomADe_EAS_AF'] = vcf['csq'].str.split('|').str[52]
vcf['gnomADe_FIN_AF'] = vcf['csq'].str.split('|').str[53]
vcf['gnomADe_NFE_AF'] = vcf['csq'].str.split('|').str[54]
vcf['gnomADe_OTH_AF'] = vcf['csq'].str.split('|').str[55]
vcf['gnomADe_SAS_AF'] = vcf['csq'].str.split('|').str[56]
vcf['gnomADg_AF'] = vcf['csq'].str.split('|').str[57]
vcf['gnomADg_AFR_AF'] = vcf['csq'].str.split('|').str[58]
vcf['gnomADg_AMI_AF'] = vcf['csq'].str.split('|').str[59]
vcf['gnomADg_AMR_AF'] = vcf['csq'].str.split('|').str[60]
vcf['gnomADg_ASJ_AF'] = vcf['csq'].str.split('|').str[61]
vcf['gnomADg_EAS_AF'] = vcf['csq'].str.split('|').str[62]
vcf['gnomADg_FIN_AF'] = vcf['csq'].str.split('|').str[63]
vcf['gnomADg_MID_AF'] = vcf['csq'].str.split('|').str[64]
vcf['gnomADg_NFE_AF'] = vcf['csq'].str.split('|').str[65]
vcf['gnomADg_OTH_AF'] = vcf['csq'].str.split('|').str[66]
vcf['gnomADg_SAS_AF'] = vcf['csq'].str.split('|').str[67]
vcf['MAX_AF'] = vcf['csq'].str.split('|').str[68]
vcf['MAX_AF_POPS'] = vcf['csq'].str.split('|').str[69]

print('Required columns extraction completed')

############################################### Protein Position and Amino Acid Change ##################################

vcf['Protein Position and Amino Acid'] = vcf['Amino_acids'].str[0] + vcf['Protein_position'] + np.where(vcf['Amino_acids'].str[-1] == vcf['Amino_acids'].str[0], '', vcf['Amino_acids'].str[-1])

############################################### HGVSc AND HGVSp TRANSCRIPTS EXTRACTION ###################################

vcf[['HGVSc', 'HGVSc (Transcript)']] = vcf['HGVSC'].str.split(':', 1, expand=True)
vcf[['HGVSp', 'HGVSp (Transcript)']] = vcf['HGVSP'].str.split(':', 1, expand=True)
vcf_final = vcf.copy()

print('Protein_HGVSc_HGVSp_extraction completed')

############################################### REMOVING THE ["not_specified", "not_provided"] FROM THE COLUMNS ##########

# Define the terms to remove
remove_terms = set(["not_specified", "not_provided"])

# Apply the filtering operation to 'Column1' only
vcf_final['ClinVar_CLNDN'] = vcf_final['ClinVar_CLNDN'].apply(lambda row: "&".join(
    [term for term in row.split("&") if term not in remove_terms]
    ) if isinstance(row, str) and not all(term in remove_terms for term in row.split("&")) else row)


vcf_final['CLIN_SIG'] = vcf_final['CLIN_SIG'].apply(lambda row: "&".join(
    [term for term in row.split("&") if term not in remove_terms]
    ) if isinstance(row, str) and not all(term in remove_terms for term in row.split("&")) else row)


vcf_final['ClinVar_CLNREVSTAT'] = vcf_final['ClinVar_CLNREVSTAT'].apply(lambda row: "&".join(
    [term for term in row.split("&") if term not in remove_terms]
    ) if isinstance(row, str) and not all(term in remove_terms for term in row.split("&")) else row)

print('"not_specified", "not_provided" completed')

#################################################### REMOVING THE UNNESESSERY CHARACTERS FROM THE COLUMNS ###################

vcf_final = vcf_final.astype(str).applymap(lambda x: x.replace('&', ',').replace('_', ' '))

############################################ CONSEQUENCE SCORES AND IMPACT SCORES MAPPING ###################################

vcf_final['consequence'] = vcf_final['Consequence'].str.split(',').str[0]

df_1 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Madhu_folder_04_07_2023/kidney_health_final.vcf/consequence.xlsx')

merged_1 = pd.merge(vcf_final, df_1, on='consequence', how='left', sort=False)

df_2 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Madhu_folder_04_07_2023/kidney_health_final.vcf/IMPACT.xlsx')

merged_2 = pd.merge(merged_1, df_2, on = 'IMPACT', how='left', sort=False)

print('Scores added')

############################################# CONDITION GENES MAPPING TO THE MAIN VCF ######################################

df_gene = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/Park_cere_genes.xlsx')

merged_2['Gene Match'] = 'No'
merged_2['Matched_Gene'] = ''
    
# Iterate through each gene in vcf['Gene']
for index, genes in merged_2['Gene Name'].iteritems():
    if isinstance(genes, str):
        gene_list = genes.split(',')
        for gene in gene_list:
            if gene in df_gene['Gene Name'].values:
                merged_2.at[index, 'Gene Match'] = 'Yes'
                merged_2.at[index, 'Matched_Gene'] = gene
                break
    
df_gene = df_gene.rename({'Gene Name':'Matched_Gene', 'Gene_Match':'Gene Match'}, axis=1)

merged_2 = merged_2.drop(columns=['Gene Match'], axis=1)

merged_2 = pd.merge(merged_2, df_gene, on= 'Matched_Gene', how = 'left', sort = False)
merged_2['Gene Match'] = merged_2['Gene Match'].fillna('No')

    
df_3 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/parkin_Cerebellarataxia_lit_pos.xlsx')


merged_2['POS'] = merged_2['POS'].astype('int64')
df_3['POS'] = df_3['POS'].astype('int64')


merged_3 = pd.merge(merged_2, df_3, on=['CHROM', 'POS'], how='left', sort=False)
merged_3['Literature'] = merged_3['Literature'].fillna('No')

merged_3 = merged_3[['Gene Name', 'Gene Match', 'rsID', 'Literature', 'CHROM', 'POS', 'REF', 'ALT', 'Zygosity',
       'Consequence', 'Consequence_score', 'IMPACT', 'IMPACT_score',
       'ClinVar_CLNDN', 'CLIN_SIG', 'ClinVar_CLNREVSTAT', 'ClinVar', 'HGVSc',
       'HGVSc (Transcript)', 'HGVSp', 'HGVSp (Transcript)', 'GT', 'GQ', 'SDP',
       'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RDF', 'RDR', 'ADF', 'ADR', 'SIFT',
       'PolyPhen', 'AF', 'AFR_AF', 'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF',
       'gnomADe_AF', 'gnomADe_AFR_AF', 'gnomADe_AMR_AF', 'gnomADe_ASJ_AF',
       'gnomADe_EAS_AF', 'gnomADe_FIN_AF', 'gnomADe_NFE_AF', 'gnomADe_OTH_AF',
       'gnomADe_SAS_AF', 'gnomADg_AF', 'gnomADg_AFR_AF', 'gnomADg_AMI_AF',
       'gnomADg_AMR_AF', 'gnomADg_ASJ_AF', 'gnomADg_EAS_AF', 'gnomADg_FIN_AF',
       'gnomADg_MID_AF', 'gnomADg_NFE_AF', 'gnomADg_OTH_AF', 'gnomADg_SAS_AF',
       'MAX_AF', 'MAX_AF_POPS', 'BIOTYPE', 'EXON', 'INTRON',
       'Protein Position and Amino Acid', 'Codons', 'STRAND', 'PUBMED']]
merged_3

#merged_3.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Processed_vcf_files/KHAIGHGPTTL207_depth_vcf_processed.xlsx', index=False)

In [56]:
# Set the folder containing your Excel files
folder_path = r'C:/Users/GenepoweRx_Madhu/Downloads/condition_specific/Condition_specific_all_lit_genes/Final_lit/last conditions/'

# Get a list of all Excel files in the folder
excel_files = [file for file in os.listdir(folder_path) if file.endswith('.xlsx')]

# Initialize an empty DataFrame to store the concatenated data
concatenated_data = pd.DataFrame()

# Iterate through each Excel file and concatenate it row by row
for excel_file in excel_files:
    # Load the Excel file into a DataFrame
    file_path = os.path.join(folder_path, excel_file)
    df = pd.read_excel(file_path)
    
    # Concatenate the data row by row
    concatenated_data = pd.concat([concatenated_data, df], axis=0, ignore_index=True)

# Save the concatenated data to a new Excel file
output_file_path = r'C:/Users/GenepoweRx_Madhu/Downloads/condition_specific/Condition_specific_all_lit_genes/Final_lit/last conditions/output_concatenated.xlsx'
concatenated_data.to_excel(output_file_path, index=False)

print(f"Concatenated data saved to {output_file_path}")


Concatenated data saved to C:/Users/GenepoweRx_Madhu/Downloads/condition_specific/Condition_specific_all_lit_genes/Final_lit/last conditions/output_concatenated.xlsx


In [57]:
df_new =  pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/condition_specific/Condition_specific_all_lit_genes/Final_lit/last conditions/output_concatenated.xlsx')
df_new

,Gene,CHROM,POS,REF,ALT,Literature
0,DNAH5,chr5,13716583,C,T,Allergies
1,DNAH5,chr5,13776446,TA,T,Allergies
2,DNAH5,chr5,13770943,GC,G,Allergies
3,HYDIN,chr16,71018411,T,C,Allergies
4,HYDIN,chr16,70959688,C,T,Allergies
...,...,...,...,...,...,...
10434,CEL,chr9,133071688,GTGACTCTGAGGCTGCCCCTGTGCCCCCCACAGA,G,Type1_Diabetes
10435,CCR5,chr3,46370444,A,G,Type1_Diabetes
10436,CCR5,chr3,46373455,AGTCAGTATCAATTCTGGAAGAATTTCCAGACA,A,Type1_Diabetes
10437,SH2B3,chr12,111446804,T,C,Type1_Diabetes


In [59]:
result_1 = df_new.groupby(['Gene', 'CHROM', 'POS', 'REF', 'ALT'])['Literature'].agg(lambda x: ', '.join(x.unique())).reset_index()
result_1

,Gene,CHROM,POS,REF,ALT,Literature
0,AAGAB,chr15,67231823,TCACT,T,Skinhealth
1,AAGAB,chr15,67236695,ATTTG,A,Skinhealth
2,AAGAB,chr15,67236718,TTGAT,T,Skinhealth
3,ABCA1,chr9,104794495,T,G,Cardiac
4,ABCA1,chr9,104799819,A,G,Cholesterol
...,...,...,...,...,...,...
8364,WSF1,chr4,6291188,A,G,Diabetes
8365,WSF1,chr4,6293966,C,G,Diabetes
8366,WSF1,chr4,6301627,G,A,Diabetes
8367,ZNF804A,chr2,184913701,A,C,Schizophrenia


In [60]:
result_1.Literature.value_counts()

Cardiac                              1569
Diabetes, Mody                       1411
Skinhealth                           1319
Diabetes                              838
Cholesterol                           828
Parkinson                             802
Dementia                              219
Thyroid                               191
Pancreatitis                          188
Diabetes, Type1_Diabetes              177
Mody                                  145
Hypertension                          136
Diabetes, Mody, Type1_Diabetes        127
Obesity                               124
Migranes                               79
Renal_stones                           74
Schizophrenia                          46
Fatty_liver                            25
Cardiac, Hypertension                  16
Cardiac, Cholesterol                   12
Migranes, Parkinson                    11
Allergies                               8
Gall_stones                             8
Dementia, Parkinson               

In [62]:
result_1.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/condition_specific/Condition_specific_all_lit_genes/Final_lit/last conditions/new_final_output_concatenated.xlsx', index=False)

In [64]:
import pandas as pd

# Create dataset1
data1 = {
    'genes': ['ABCC8,INS', 'BRCA1', 'BRCA1,BRCA2', 'HNF1A', 'HNF1A,SLCOB', 'BRCA1,ABCC8,INS']
}
dataset1 = pd.DataFrame(data1)

# Create dataset2
data2 = {
    'genes': ['ABCC8', 'BRCA1', 'BRCA2', 'HNF1A', 'SLCOB'],
    'CHROM': ['chr1', 'chr12', 'chr12', 'chr3', 'chrX'],
    'POS': [112234, 34567, 67890, 7890, 12345],
    'LIT': ['Diabetes', 'Thyroid', 'Breast Cancer', 'Diabetes', 'Transporter']
}
dataset2 = pd.DataFrame(data2)
dataset2

,genes,CHROM,POS,LIT
0,ABCC8,chr1,112234,Diabetes
1,BRCA1,chr12,34567,Thyroid
2,BRCA2,chr12,67890,Breast Cancer
3,HNF1A,chr3,7890,Diabetes
4,SLCOB,chrX,12345,Transporter


In [65]:
dataset1

,genes
0,"ABCC8,INS"
1,BRCA1
2,"BRCA1,BRCA2"
3,HNF1A
4,"HNF1A,SLCOB"
5,"BRCA1,ABCC8,INS"


In [68]:
# Split the 'genes' column in dataset1 and create a new DataFrame
genes_split = dataset1['genes'].str.split(',', expand=True)

# Create a list of all unique genes in both datasets
all_genes = set(gene for gene_list in genes_split.values.flatten() if gene_list is not None for gene in gene_list)
all_genes.update(dataset2['genes'].unique())

# Filter dataset1 rows where any gene matches genes in dataset2
filtered_dataset1 = dataset1[genes_split.apply(lambda row: any(gene in all_genes for gene in row) if row is not None else False, axis=1)]

# Merge the filtered rows from dataset1 with dataset2 based on the 'genes' column
merged_data = pd.merge(filtered_dataset1, dataset2, on='genes', how='inner')

# Output the merged data
merged_data

,genes,CHROM,POS,LIT
0,BRCA1,chr12,34567,Thyroid
1,HNF1A,chr3,7890,Diabetes


In [8]:
import pandas as pd
import numpy as np

# Create a sample DataFrame
data = {'Value': [10, 25, 35, 50, 65, 75, 90]}
df = pd.DataFrame(data)

# Define the cutoff points
cutoffs = [0, 30, 60, np.inf]  # You can adjust these cutoffs as needed

# Create labels for the cutoff ranges
labels = ['Low', 'Medium', 'High']

# Use pd.cut() to categorize the data into cutoff ranges
df['Category'] = pd.cut(df['Value'], bins=cutoffs, labels=labels)

# Display the updated DataFrame
df

,Value,Category
0,10,Low
1,25,Low
2,35,Medium
3,50,Medium
4,65,High
5,75,High
6,90,High


# code

In [56]:
import pandas as pd

# Read the VCF file into a Pandas DataFrame (skip first 80 rows as shown in a previous answer)
vcf = pd.read_csv(
    r'C:/Users/GenepoweRx_Madhu/Downloads/718_final.vcf',
    comment='#',
    sep='\t',
    header=None,
    low_memory=False,
    skiprows=80
)

# Assign appropriate column names
vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']
vcf["Gene"] = vcf["INFO"].str.extract('GENEINFO=(?P<GENEINFO>.+?);')
vcf['Gene'] = vcf['Gene'].apply(lambda x: ','.join(set([segment.split(':')[0] for segment in x.split('|')])) if pd.notnull(x) else '')
df_gene = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/selective_genes.xlsx')

vcf['Matched_gene'] = 'No'

# Iterate through each gene in df1
for genes in vcf['Gene']:
    if isinstance(genes, str):  # Check if the gene value is a non-null string
        gene_list = genes.split(',')  # Split the genes by comma to create a list
        match = any(gene in df_gene['Gene'].values for gene in gene_list)  # Check if any gene in the list exists in df2
        if match:
            vcf.loc[vcf['Gene'] == genes, 'Matched_gene'] = 'Yes'
            
vcf = vcf[vcf['Matched_gene'] == 'Yes']

sample_cols = vcf['SAMPLE'].str.split(':', expand=True)
sample_cols.columns = ['GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RBQ', 'ABQ', 'RDF', 'RDR', 'ADF', 'ADR']

# Assign the values to the newly created columns
vcf = pd.concat([vcf, sample_cols], axis=1)

# Create empty columns
columns = ['ADP', 'WT', 'HET', 'HOM', 'NC', 'CDA', 'OTH', 'S3D', 'WTD', 'dbSNPBuildID', 'SLO',
           'NSF', 'R3', 'R5', 'NSN', 'NSM', 'G5A', 'COMMON', 'RS', 'RV', 'TPA', 'CFL', 'GNO',
           'VLD', 'ASP', 'ASS', 'Ref', 'U3', 'U5', 'TOPMED', 'WGT', 'MTP', 'LSD', 'NOC',
           'DSS', 'SYN', 'KGPhase3', 'CAF', 'VC', 'MUT', 'KGPhase1', 'NOV', 'VP', 'SAO',
           'GENEINFO', 'INT', 'G5', 'OM', 'PMC', 'SSR', 'RSPOS', 'HD', 'PM']

for col in columns:
    vcf[col] = ''

# Populate columns based on 'INFO' values
for i, row in vcf.iterrows():
    info = row['INFO']
    items = info.split(';')
    for item in items:
        key_value = item.split('=')
        key = key_value[0]
        if key in columns:
            if len(key_value) > 1:
                value = key_value[1]
                vcf.at[i, key] = f"{key}={value}"

# Extract the numeric part of the HET column and convert it to an integer
vcf['HET_new'] = vcf['HET'].str.extract('(\d+)').astype(int)

# Create the Zygosity column based on the HET column
vcf['Zygosity'] = vcf['HET_new'].apply(lambda x: 'Heterozygous' if x == 1 else 'Homozygous')

# Extract the CSQ field from the INFO column
vcf['CSQ'] = vcf['INFO'].str.extract('CSQ=([^;]+)')

# Split the CSQ field into individual transcripts
transcripts = vcf['CSQ'].str.split(',')

CSQ_columns = ["Allele", "Consequence", "IMPACT", "SYMBOL", "Gene", "Feature_type", "Feature", "BIOTYPE", "EXON", "INTRON", "HGVSc", "HGVSp", "cDNA_position", "CDS_position", "Protein_position", "Amino_acids", "Codons", "Existing_variation", "DISTANCE", "STRAND", "FLAGS", "SYMBOL_SOURCE", "HGNC_ID", "SOURCE", "ClinVar", "ClinVar_CLNSIG", "ClinVar_CLNREVSTAT", "ClinVar_CLNDN"]

# Initialize an empty DataFrame to store the extracted values
extracted_df = pd.DataFrame(columns=CSQ_columns)

# Helper function to remove leading and trailing commas
def remove_leading_trailing_commas(s):
    if isinstance(s, str):
        return s.strip(',')
    return s

# Iterate over each transcript and extract values into separate columns
for transcript in transcripts:
    values = {}
    for entry in transcript:
        entry = remove_leading_trailing_commas(entry)  # Remove leading and trailing commas
        parts = entry.split('|')
        for i, column_name in enumerate(CSQ_columns):
            if i < len(parts):
                if column_name in values:
                    values[column_name] += ',' + parts[i]
                else:
                    values[column_name] = parts[i]
    extracted_df = extracted_df.append(values, ignore_index=True)

# Apply set to each cell to get unique values
unique_extracted_df = extracted_df.applymap(lambda x: ','.join(set(x.split(',')) if isinstance(x, str) else ''))

# Remove leading and trailing commas in the entire DataFrame
unique_extracted_df = unique_extracted_df.applymap(remove_leading_trailing_commas)

# Reset the index of both DataFrames
vcf.reset_index(drop=True, inplace=True)
unique_extracted_df.reset_index(drop=True, inplace=True)
combined_df = pd.concat([vcf, unique_extracted_df], axis=1)
columns_to_drop = [7, 8, 9, 11, 79, 81]
combined_df = combined_df.drop(columns=combined_df.columns[columns_to_drop])
combined_df.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/biotype_final_csq_specific_gene.xlsx', index = False)
combined_df

#CSQ_columns = [
#    "Allele", "Consequence", "IMPACT", "SYMBOL", "Gene", "Feature_type", "Feature",
#    "BIOTYPE", "EXON", "INTRON", "HGVSc", "HGVSp", "cDNA_position", "CDS_position",
#    "Protein_position", "Amino_acids", "Codons", "Existing_variation", "DISTANCE",
#    "STRAND", "FLAGS", "VARIANT_CLASS", "SYMBOL_SOURCE", "HGNC_ID", "CANONICAL",
#    "MANE_SELECT", "MANE_PLUS_CLINICAL", "TSL", "APPRIS", "CCDS", "ENSP", "SWISSPROT",
#    "TREMBL", "UNIPARC", "UNIPROT_ISOFORM", "SOURCE", "GENE_PHENO", "SIFT", "PolyPhen",
#    "DOMAINS", "miRNA", "HGVS_OFFSET", "AF", "AFR_AF", "AMR_AF", "EAS_AF", "EUR_AF",
#    "SAS_AF", "gnomADe_AF", "gnomADe_AFR_AF", "gnomADe_AMR_AF", "gnomADe_ASJ_AF",
#    "gnomADe_EAS_AF", "gnomADe_FIN_AF", "gnomADe_NFE_AF", "gnomADe_OTH_AF", "gnomADe_SAS_AF",
#    "gnomADg_AF", "gnomADg_AFR_AF", "gnomADg_AMI_AF", "gnomADg_AMR_AF", "gnomADg_ASJ_AF",
#    "gnomADg_EAS_AF", "gnomADg_FIN_AF", "gnomADg_MID_AF", "gnomADg_NFE_AF", "gnomADg_OTH_AF",
#    "gnomADg_SAS_AF", "MAX_AF", "MAX_AF_POPS", "CLIN_SIG", "SOMATIC", "PHENO", "PUBMED",
#    "MOTIF_NAME", "MOTIF_POS", "HIGH_INF_POS", "MOTIF_SCORE_CHANGE", "TRANSCRIPTION_FACTORS",
#    "ClinVar", "ClinVar_CLNSIG", "ClinVar_CLNREVSTAT", "ClinVar_CLNDN"]

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,Gene,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RBQ,ABQ,RDF,RDR,ADF,ADR,ADP,WT,HET,HOM,NC,CDA,OTH,S3D,WTD,dbSNPBuildID,SLO,NSF,R3,R5,NSN,NSM,G5A,COMMON,RS,RV,TPA,CFL,GNO,VLD,ASP,ASS,Ref,U3,U5,TOPMED,WGT,MTP,LSD,NOC,DSS,SYN,KGPhase3,CAF,VC,MUT,KGPhase1,NOV,VP,SAO,GENEINFO,INT,G5,OM,PMC,SSR,RSPOS,HD,PM,Zygosity,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,SYMBOL_SOURCE,HGNC_ID,SOURCE,ClinVar,ClinVar_CLNSIG,ClinVar_CLNREVSTAT,ClinVar_CLNDN
0,chr1,69511,rs2691305,A,G,.,PASS,OR4F5,1/1,255,90,90,0,90,100%,1.0988E-53,0,54,0,0,71,19,ADP=90,WT=0,HET=0,HOM=1,NC=0,,,,,dbSNPBuildID=100,,,,,,,,,RS=2691305,,,,,,,,,,,"TOPMED=0.12468941131498470,0.87531058868501529,.",WGT=1,,,,,,,,VC=SNV,,,,VP=0x050100000a05050116000100,SAO=0,GENEINFO=OR4F5:79501,,,,,SSR=0,RSPOS=69511,,,Homozygous,G,missense_variant,MODERATE,OR4F5,ENSG00000186092,Transcript,ENST00000641515,protein_coding,3/3,,,,544,484,162,T/A,Aca/Gca,,,1,,HGNC,HGNC:14825,,,,,
1,chr1,69897,rs200676709,T,C,.,PASS,OR4F5,0/1,72,96,96,74,22,22.92%,6.1288E-8,54,53,51,23,15,7,ADP=96,WT=0,HET=1,HOM=0,NC=0,,,,,dbSNPBuildID=137,,,,,,,,COMMON=1,RS=200676709,,,,,,,,,,,"TOPMED=0.71955434505606523,0.28044565494393476",WGT=1,,,,,,,"CAF=0.3119,0.6881",VC=SNV,,,,VP=0x050300000305170026000100,SAO=0,GENEINFO=OR4F5:79501,,,,,SSR=0,RSPOS=69897,,,Heterozygous,C,synonymous_variant,LOW,OR4F5,ENSG00000186092,Transcript,ENST00000641515,protein_coding,3/3,,,,930,870,290,S,tcT/tcC,,,1,,HGNC,HGNC:14825,,,,,
2,chr1,97078196,rs291593,G,A,.,PASS,DPYD,1/1,41,8,8,0,8,100%,7.77E-5,0,52,0,0,8,0,ADP=8,WT=0,HET=0,HOM=1,NC=0,,,,,dbSNPBuildID=79,,,,,,,,COMMON=1,RS=291593,,,,,,,,,,,"TOPMED=0.70939252038735983,0.29060747961264016",WGT=1,,,,,,,"CAF=0.615,0.385",VC=SNV,,,,VP=0x05016880000517053e100100,SAO=1,GENEINFO=DPYD:1806,,,,,SSR=0,RSPOS=97078196,,,Homozygous,A,3_prime_UTR_variant,MODIFIER,DPYD,ENSG00000188641,Transcript,ENST00000370192,protein_coding,23/23,,,,3970,,,,,,,-1,,HGNC,HGNC:3012,,100064,Benign,criteria_provided&_single_submitter,Dihydropyrimidine_dehydrogenase_deficiency&not...
3,chr1,97078390,rs886046571,G,A,.,PASS,DPYD,0/1,31,12,12,4,8,66.67%,6.7304E-4,34,60,2,2,7,1,ADP=12,WT=0,HET=1,HOM=0,NC=0,,,,,dbSNPBuildID=149,,,,,,,,,RS=886046571,,,,,,,,,,,"TOPMED=0.99979294087665647,0.00020705912334352",WGT=1,,,,,,,,VC=SNV,,,,VP=0x050060800005000002100100,SAO=1,GENEINFO=DPYD:1806,,,,,SSR=0,RSPOS=97078390,,,Heterozygous,A,3_prime_UTR_variant,MODIFIER,DPYD,ENSG00000188641,Transcript,ENST00000370192,protein_coding,23/23,,,,3776,,,,,,,-1,,HGNC,HGNC:3012,,298274,Uncertain_significance,criteria_provided&_single_submitter,Dihydropyrimidine_dehydrogenase_deficiency
4,chr1,97078403,rs1042482,C,T,.,PASS,DPYD,1/1,41,8,8,0,8,100%,7.77E-5,0,48,0,0,6,2,ADP=8,WT=0,HET=0,HOM=1,NC=0,,,,,dbSNPBuildID=86,,,,,,,,COMMON=1,RS=1042482,,,,,,,,,,,"TOPMED=0.94017584097859327,0.05982415902140672",WGT=1,,,,,,,"CAF=0.9105,0.08946",VC=SNV,,,,VP=0x050168800005150536100100,SAO=1,GENEINFO=DPYD:1806,,,,,SSR=0,RSPOS=97078403,,,Homozygous,T,3_prime_UTR_variant,MODIFIER,DPYD,ENSG00000188641,Transcript,ENST00000370192,protein_coding,23/23,,,,3763,,,,,,,-1,,HGNC,HGNC:3012,,100067,Benign,criteria_provided&_single_submitter,Dihydropyrimidine_dehydrogenase_deficiency&not...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,chr7,117590591,rs4148711,T,A,.,PASS,CFTR,1/1,81,15,15,0,15,100%,6.4467E-9,0,51,0,0,6,9,ADP=15,WT=0,HET=0,HOM=1,NC=0,,,,,dbSNPBuildID=110,,,,,,,,COMMON=1,RS=4148711,,,,,,,,,,,"TOPMED=0.56985059887869520,0.4301494011

In [50]:
columns_to_drop = [7, 8, 9, 11, 79, 81]
combined_df = combined_df.drop(columns=combined_df.columns[columns_to_drop])
combined_df

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,Gene,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RBQ,ABQ,RDF,RDR,ADF,ADR,ADP,WT,HET,HOM,NC,CDA,OTH,S3D,WTD,dbSNPBuildID,SLO,NSF,R3,R5,NSN,NSM,G5A,COMMON,RS,RV,TPA,CFL,GNO,VLD,ASP,ASS,Ref,U3,U5,TOPMED,WGT,MTP,LSD,NOC,DSS,SYN,KGPhase3,CAF,VC,MUT,KGPhase1,NOV,VP,SAO,GENEINFO,INT,G5,OM,PMC,SSR,RSPOS,HD,PM,ClinVar,ClinVar_CLNSIG,Zygosity,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,SYMBOL_SOURCE,HGNC_ID,SOURCE,ClinVar,ClinVar_CLNSIG,ClinVar_CLNREVSTAT,ClinVar_CLNDN
0,chr1,69511,rs2691305,A,G,.,PASS,OR4F5,1/1,255,90,90,0,90,100%,1.0988E-53,0,54,0,0,71,19,ADP=90,WT=0,HET=0,HOM=1,NC=0,,,,,dbSNPBuildID=100,,,,,,,,,RS=2691305,,,,,,,,,,,"TOPMED=0.12468941131498470,0.87531058868501529,.",WGT=1,,,,,,,,VC=SNV,,,,VP=0x050100000a05050116000100,SAO=0,GENEINFO=OR4F5:79501,,,,,SSR=0,RSPOS=69511,,,,,Homozygous,G,missense_variant,MODERATE,OR4F5,ENSG00000186092,Transcript,ENST00000641515,protein_coding,3/3,,,,544,484,162,T/A,Aca/Gca,,,1,,HGNC,HGNC:14825,,,,,
1,chr1,69897,rs200676709,T,C,.,PASS,OR4F5,0/1,72,96,96,74,22,22.92%,6.1288E-8,54,53,51,23,15,7,ADP=96,WT=0,HET=1,HOM=0,NC=0,,,,,dbSNPBuildID=137,,,,,,,,COMMON=1,RS=200676709,,,,,,,,,,,"TOPMED=0.71955434505606523,0.28044565494393476",WGT=1,,,,,,,"CAF=0.3119,0.6881",VC=SNV,,,,VP=0x050300000305170026000100,SAO=0,GENEINFO=OR4F5:79501,,,,,SSR=0,RSPOS=69897,,,,,Heterozygous,C,synonymous_variant,LOW,OR4F5,ENSG00000186092,Transcript,ENST00000641515,protein_coding,3/3,,,,930,870,290,S,tcT/tcC,,,1,,HGNC,HGNC:14825,,,,,
2,chr1,97078196,rs291593,G,A,.,PASS,DPYD,1/1,41,8,8,0,8,100%,7.77E-5,0,52,0,0,8,0,ADP=8,WT=0,HET=0,HOM=1,NC=0,,,,,dbSNPBuildID=79,,,,,,,,COMMON=1,RS=291593,,,,,,,,,,,"TOPMED=0.70939252038735983,0.29060747961264016",WGT=1,,,,,,,"CAF=0.615,0.385",VC=SNV,,,,VP=0x05016880000517053e100100,SAO=1,GENEINFO=DPYD:1806,,,,,SSR=0,RSPOS=97078196,,,,,Homozygous,A,3_prime_UTR_variant,MODIFIER,DPYD,ENSG00000188641,Transcript,ENST00000370192,protein_coding,23/23,,,,3970,,,,,,,-1,,HGNC,HGNC:3012,,100064,Benign,criteria_provided&_single_submitter,Dihydropyrimidine_dehydrogenase_deficiency&not...
3,chr1,97078390,rs886046571,G,A,.,PASS,DPYD,0/1,31,12,12,4,8,66.67%,6.7304E-4,34,60,2,2,7,1,ADP=12,WT=0,HET=1,HOM=0,NC=0,,,,,dbSNPBuildID=149,,,,,,,,,RS=886046571,,,,,,,,,,,"TOPMED=0.99979294087665647,0.00020705912334352",WGT=1,,,,,,,,VC=SNV,,,,VP=0x050060800005000002100100,SAO=1,GENEINFO=DPYD:1806,,,,,SSR=0,RSPOS=97078390,,,,,Heterozygous,A,3_prime_UTR_variant,MODIFIER,DPYD,ENSG00000188641,Transcript,ENST00000370192,protein_coding,23/23,,,,3776,,,,,,,-1,,HGNC,HGNC:3012,,298274,Uncertain_significance,criteria_provided&_single_submitter,Dihydropyrimidine_dehydrogenase_deficiency
4,chr1,97078403,rs1042482,C,T,.,PASS,DPYD,1/1,41,8,8,0,8,100%,7.77E-5,0,48,0,0,6,2,ADP=8,WT=0,HET=0,HOM=1,NC=0,,,,,dbSNPBuildID=86,,,,,,,,COMMON=1,RS=1042482,,,,,,,,,,,"TOPMED=0.94017584097859327,0.05982415902140672",WGT=1,,,,,,,"CAF=0.9105,0.08946",VC=SNV,,,,VP=0x050168800005150536100100,SAO=1,GENEINFO=DPYD:1806,,,,,SSR=0,RSPOS=97078403,,,,,Homozygous,T,3_prime_UTR_variant,MODIFIER,DPYD,ENSG00000188641,Transcript,ENST00000370192,protein_coding,23/23,,,,3763,,,,,,,-1,,HGNC,HGNC:3012,,100067,Benign,criteria_provided&_single_submitter,Dihydropyrimidine_dehydrogenase_deficiency&not...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,chr7,117590591,rs4148711,T,A,.,PASS,CFTR,1/1,81,15,15,0,15,100%,6.4467E-9,0,51,0,0,6,9,ADP=15,WT=0,HET=0,HOM=1,NC=0,,,,,dbSNPBuildID=110,,,,,,,,COMMON=1,RS=4148711,,,,,,,,,,

In [53]:
print(list(combined_df.columns))

['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE', 'Gene', 'Matched_gene', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RBQ', 'ABQ', 'RDF', 'RDR', 'ADF', 'ADR', 'ADP', 'WT', 'HET', 'HOM', 'NC', 'CDA', 'OTH', 'S3D', 'WTD', 'dbSNPBuildID', 'SLO', 'NSF', 'R3', 'R5', 'NSN', 'NSM', 'G5A', 'COMMON', 'RS', 'RV', 'TPA', 'CFL', 'GNO', 'VLD', 'ASP', 'ASS', 'Ref', 'U3', 'U5', 'TOPMED', 'WGT', 'MTP', 'LSD', 'NOC', 'DSS', 'SYN', 'KGPhase3', 'CAF', 'VC', 'MUT', 'KGPhase1', 'NOV', 'VP', 'SAO', 'GENEINFO', 'INT', 'G5', 'OM', 'PMC', 'SSR', 'RSPOS', 'HD', 'PM', 'HET_new', 'Zygosity', 'CSQ', 'Allele', 'Consequence', 'IMPACT', 'SYMBOL', 'Gene', 'Feature_type', 'Feature', 'BIOTYPE', 'EXON', 'INTRON', 'HGVSc', 'HGVSp', 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'DISTANCE', 'STRAND', 'FLAGS', 'SYMBOL_SOURCE', 'HGNC_ID', 'SOURCE', 'ClinVar', 'ClinVar_CLNSIG', 'ClinVar_CLNREVSTAT', 'ClinVar_CLNDN']


In [36]:
# Extract the CSQ field from the INFO column
vcf['CSQ'] = vcf['INFO'].str.extract('CSQ=([^;]+)')

# Split the CSQ field into individual transcripts
transcripts = vcf['CSQ'].str.split(',')

# Define the column names
#column_names = [
#    "Allele", "Consequence", "IMPACT", "SYMBOL", "Gene", "Feature_type", "Feature",
#    "BIOTYPE", "EXON", "INTRON", "HGVSc", "HGVSp", "cDNA_position", "CDS_position",
#    "Protein_position", "Amino_acids", "Codons", "Existing_variation", "DISTANCE",
#    "STRAND", "FLAGS", "VARIANT_CLASS", "SYMBOL_SOURCE", "HGNC_ID", "CANONICAL",
#    "MANE_SELECT", "MANE_PLUS_CLINICAL", "TSL", "APPRIS", "CCDS", "ENSP", "SWISSPROT",
#    "TREMBL", "UNIPARC", "UNIPROT_ISOFORM", "SOURCE", "GENE_PHENO", "SIFT", "PolyPhen",
#    "DOMAINS", "miRNA", "HGVS_OFFSET", "AF", "AFR_AF", "AMR_AF", "EAS_AF", "EUR_AF",
#    "SAS_AF", "gnomADe_AF", "gnomADe_AFR_AF", "gnomADe_AMR_AF", "gnomADe_ASJ_AF",
#    "gnomADe_EAS_AF", "gnomADe_FIN_AF", "gnomADe_NFE_AF", "gnomADe_OTH_AF", "gnomADe_SAS_AF",
#    "gnomADg_AF", "gnomADg_AFR_AF", "gnomADg_AMI_AF", "gnomADg_AMR_AF", "gnomADg_ASJ_AF",
#    "gnomADg_EAS_AF", "gnomADg_FIN_AF", "gnomADg_MID_AF", "gnomADg_NFE_AF", "gnomADg_OTH_AF",
#    "gnomADg_SAS_AF", "MAX_AF", "MAX_AF_POPS", "CLIN_SIG", "SOMATIC", "PHENO", "PUBMED",
#    "MOTIF_NAME", "MOTIF_POS", "HIGH_INF_POS", "MOTIF_SCORE_CHANGE", "TRANSCRIPTION_FACTORS",
#    "ClinVar", "ClinVar_CLNSIG", "ClinVar_CLNREVSTAT", "ClinVar_CLNDN"]

column_names = ["Allele", "Consequence", "IMPACT", "SYMBOL", "Gene", "Feature_type", "Feature", "BIOTYPE", "EXON", "INTRON", "HGVSc", "HGVSp", "cDNA_position", "CDS_position", "Protein_position", "Amino_acids", "Codons", "Existing_variation", "DISTANCE", "STRAND", "FLAGS", "SYMBOL_SOURCE", "HGNC_ID", "SOURCE", "ClinVar", "ClinVar_CLNSIG", "ClinVar_CLNREVSTAT", "ClinVar_CLNDN"]

# Initialize an empty DataFrame to store the extracted values
extracted_df = pd.DataFrame(columns=column_names)

# Iterate over each transcript and extract values into separate columns
for transcript in transcripts:
    values = {}
    for entry in transcript:
        if isinstance(entry, str):
            parts = entry.split('|')
            for i, column_name in enumerate(column_names):
                if i < len(parts):
                    if column_name in values:
                        values[column_name] += ',' + parts[i]
                    else:
                        values[column_name] = parts[i]
    extracted_df = extracted_df.append(values, ignore_index=True)

# Apply set to each cell to get unique values
unique_extracted_df = extracted_df.applymap(lambda x: ','.join(set(x.split(',')) if isinstance(x, str) else ''))



# Display the DataFrame with unique values
unique_extracted_df

,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,SYMBOL_SOURCE,HGNC_ID,SOURCE,ClinVar,ClinVar_CLNSIG,ClinVar_CLNREVSTAT,ClinVar_CLNDN
0,G,missense_variant,MODERATE,OR4F5,ENSG00000186092,Transcript,ENST00000641515,protein_coding,3/3,,,,544,484,162,T/A,Aca/Gca,,,1,,HGNC,HGNC:14825,,,,,
1,C,synonymous_variant,LOW,OR4F5,ENSG00000186092,Transcript,ENST00000641515,protein_coding,3/3,,,,930,870,290,S,tcT/tcC,,,1,,HGNC,HGNC:14825,,,,,
2,A,3_prime_UTR_variant,MODIFIER,DPYD,ENSG00000188641,Transcript,ENST00000370192,protein_coding,23/23,,,,3970,,,,,,,-1,,HGNC,HGNC:3012,,100064,Benign,criteria_provided&_single_submitter,Dihydropyrimidine_dehydrogenase_deficiency&not...
3,A,3_prime_UTR_variant,MODIFIER,DPYD,ENSG00000188641,Transcript,ENST00000370192,protein_coding,23/23,,,,3776,,,,,,,-1,,HGNC,HGNC:3012,,298274,Uncertain_significance,criteria_provided&_single_submitter,Dihydropyrimidine_dehydrogenase_deficiency
4,T,3_prime_UTR_variant,MODIFIER,DPYD,ENSG00000188641,Transcript,ENST00000370192,protein_coding,23/23,,,,3763,,,,,,,-1,,HGNC,HGNC:3012,,100067,Benign,criteria_provided&_single_submitter,Dihydropyrimidine_dehydrogenase_deficiency&not...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,A,"downstream_gene_variant,intron_variant,intron_...",MODIFIER,",CFTR","ENSG00000001626,ENSG00000234001",Transcript,"ENST00000649406,ENST00000699598,ENST0000068727...","processed_transcript,processed_pseudogene,prot...",,",13/28,13/27,13/26,12/25,10/16,12/20,13/29",,,,,,,,,",4175,2842","-1,1",",cds_end_NF",",HGNC",",HGNC:1884",,818094,Benign,criteria_provided&_multiple_submitters&_no_con...,Cystic_fibrosis&not_provided
101,G,"intron_variant,synonymous_variant&NMD_transcri...","LOW,MODIFIER",CFTR,ENSG00000001626,Transcript,"ENST00000649781,ENST00000699602,ENST0000068501...","protein_coding,nonsense_mediated_decay",",15/29,15/30,15/28,14/26,14/21,15/27",",10/16",,,",2472,2644,2669,2664,2558,2537,2632,2463",",2472,2562,2379,2136",",854,824,793,712",",T","acT/acG,",,,1,",cds_end_NF",HGNC,HGNC:1884,,43577,Benign/Likely_benign,criteria_provided&_multiple_submitters&_no_con...,CFTR-related_disorders&Hereditary_pancreatitis...
102,A,"intron_variant&non_coding_transcript_variant,i...",MODIFIER,",CFTR","ENSG00000083622,ENSG00000001626",Transcript,"ENST00000649406,ENST00000699598,ENST0000068727...","nonsense_mediated_decay,protein_coding,lncRNA",,"2/2,21/26,20/25,22/28,21/28,1/6,20/20,21/27,21...",,,,,,,,,,"-1,1",",cds_start_NF,cds_end_NF",",HGNC",",HGNC:1884",,818096,Benign,criteria_provided&_multiple_submitters&_no_con...,Cystic_fibrosis&not_provided
103,A,"non_coding_transcript_exon_variant,intron_vari...","LOW,MODIFIER",CFTR,ENSG00000001626,Transcript,"ENST00000649781,ENST00000699602,ENST0000068501...","protein_coding,nonsense_mediated_decay,retaine...",",27/27,7/7,26/26,27/29,28/28,3/3,28/30",",3/3,27/28",,,",4299,1231,4490,4566,4556,4296,3900,4459,4290,...","4299,,4383,4206,3963,4389",",1463,1461,1321,1402,1433",",Q",",caG/caA",,,1,",cds_start_NF,cds_end_NF",HGNC,HGNC:1884,,93157,Benign/Likely_benign,criteria_provided&_multiple_submitters&_no_con...,CFTR-related_disorders&Hereditary_pancreatitis...


In [38]:
# Reset the index of both DataFrames
vcf.reset_index(drop=True, inplace=True)
unique_extracted_df.reset_index(drop=True, inplace=True)
combined_df = pd.concat([vcf, unique_extracted_df], axis=1)
combined_df

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE,Gene,Matched_gene,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RBQ,ABQ,RDF,RDR,ADF,ADR,ADP,WT,HET,HOM,NC,CDA,OTH,S3D,WTD,dbSNPBuildID,SLO,NSF,R3,R5,NSN,NSM,G5A,COMMON,RS,RV,TPA,CFL,GNO,VLD,ASP,ASS,Ref,U3,U5,TOPMED,WGT,MTP,LSD,NOC,DSS,SYN,KGPhase3,CAF,VC,MUT,KGPhase1,NOV,VP,SAO,GENEINFO,INT,G5,OM,PMC,SSR,RSPOS,HD,PM,ClinVar,ClinVar_CLNSIG,HET_new,Zygosity,CSQ,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,SYMBOL_SOURCE,HGNC_ID,SOURCE,ClinVar,ClinVar_CLNSIG,ClinVar_CLNREVSTAT,ClinVar_CLNDN
0,chr1,69511,rs2691305,A,G,.,PASS,ADP=90;WT=0;HET=0;HOM=1;NC=0;ASP;G5;GENEINFO=O...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:255:90:90:0:90:100%:1.0988E-53:0:54:0:0:71:19,OR4F5,Yes,1/1,255,90,90,0,90,100%,1.0988E-53,0,54,0,0,71,19,ADP=90,WT=0,HET=0,HOM=1,NC=0,,,,,dbSNPBuildID=100,,,,,,,,,RS=2691305,,,,,,,,,,,"TOPMED=0.12468941131498470,0.87531058868501529,.",WGT=1,,,,,,,,VC=SNV,,,,VP=0x050100000a05050116000100,SAO=0,GENEINFO=OR4F5:79501,,,,,SSR=0,RSPOS=69511,,,,,0,Homozygous,G|missense_variant|MODERATE|OR4F5|ENSG00000186...,G,missense_variant,MODERATE,OR4F5,ENSG00000186092,Transcript,ENST00000641515,protein_coding,3/3,,,,544,484,162,T/A,Aca/Gca,,,1,,HGNC,HGNC:14825,,,,,
1,chr1,69897,rs200676709,T,C,.,PASS,"ADP=96;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.3119,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:72:96:96:74:22:22.92%:6.1288E-8:54:53:51:2...,OR4F5,Yes,0/1,72,96,96,74,22,22.92%,6.1288E-8,54,53,51,23,15,7,ADP=96,WT=0,HET=1,HOM=0,NC=0,,,,,dbSNPBuildID=137,,,,,,,,COMMON=1,RS=200676709,,,,,,,,,,,"TOPMED=0.71955434505606523,0.28044565494393476",WGT=1,,,,,,,"CAF=0.3119,0.6881",VC=SNV,,,,VP=0x050300000305170026000100,SAO=0,GENEINFO=OR4F5:79501,,,,,SSR=0,RSPOS=69897,,,,,1,Heterozygous,C|synonymous_variant|LOW|OR4F5|ENSG00000186092...,C,synonymous_variant,LOW,OR4F5,ENSG00000186092,Transcript,ENST00000641515,protein_coding,3/3,,,,930,870,290,S,tcT/tcC,,,1,,HGNC,HGNC:14825,,,,,
2,chr1,97078196,rs291593,G,A,.,PASS,"ADP=8;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.615,0.38...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:41:8:8:0:8:100%:7.77E-5:0:52:0:0:8:0,DPYD,Yes,1/1,41,8,8,0,8,100%,7.77E-5,0,52,0,0,8,0,ADP=8,WT=0,HET=0,HOM=1,NC=0,,,,,dbSNPBuildID=79,,,,,,,,COMMON=1,RS=291593,,,,,,,,,,,"TOPMED=0.70939252038735983,0.29060747961264016",WGT=1,,,,,,,"CAF=0.615,0.385",VC=SNV,,,,VP=0x05016880000517053e100100,SAO=1,GENEINFO=DPYD:1806,,,,,SSR=0,RSPOS=97078196,,,,,0,Homozygous,A|3_prime_UTR_variant|MODIFIER|DPYD|ENSG000001...,A,3_prime_UTR_variant,MODIFIER,DPYD,ENSG00000188641,Transcript,ENST00000370192,protein_coding,23/23,,,,3970,,,,,,,-1,,HGNC,HGNC:3012,,100064,Benign,criteria_provided&_single_submitter,Dihydropyrimidine_dehydrogenase_deficiency&not...
3,chr1,97078390,rs886046571,G,A,.,PASS,ADP=12;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DPYD...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:31:12:12:4:8:66.67%:6.7304E-4:34:60:2:2:7:1,DPYD,Yes,0/1,31,12,12,4,8,66.67%,6.7304E-4,34,60,2,2,7,1,ADP=12,WT=0,HET=1,HOM=0,NC=0,,,,,dbSNPBuildID=149,,,,,,,,,RS=886046571,,,,,,,,,,,"TOPMED=0.99979294087665647,0.00020705912334352",WGT=1,,,,,,,,VC=SNV,,,,VP=0x050060800005000002100100,SAO=1,GENEINFO=DPYD:1806,,,,,SSR=0,RSPOS=97078390,,,,,1,Heterozygous,A|3_prime_UTR_variant|MODIFIER|DPYD|ENSG000001...,A,3_prime_UTR_variant,MODIFIER,DPYD,ENSG00000188641,Transcript,ENST00000370192,protein_coding,23/23,,,,3776,,,,,,,-1,,HGNC,HGNC:3012,,298274,Uncertain_significance,criteria_provided&_single_submitter,Dihydropyrimidine_dehydrogenase_deficiency
4,chr1,97078403,rs1042482,C,T,.,PASS,"ADP=8;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.9105,0.0...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:41:8:8:0:8:100%:7.77E-5:0:48:0:0:6:2,DPYD,Yes,1/1,41,8,8,0,8,100%,7.77E-5,0,48,0,0,6,2,ADP=8,WT=0,HET=0,HOM=1,NC=0,,,,,dbSNPBuildID=86,,,,,,,,COMMON=1,RS=1042482,,,,,,,,,,,"TOPMED=0.94017584

In [45]:
# Extract the CSQ field from the INFO column
vcf['CSQ'] = vcf['INFO'].str.extract('CSQ=([^;]+)')

# Split the CSQ field into individual transcripts
transcripts = vcf['CSQ'].str.split(',')

# Define the column names
#column_names = [
#    "Allele", "Consequence", "IMPACT", "SYMBOL", "Gene", "Feature_type", "Feature",
#    "BIOTYPE", "EXON", "INTRON", "HGVSc", "HGVSp", "cDNA_position", "CDS_position",
#    "Protein_position", "Amino_acids", "Codons", "Existing_variation", "DISTANCE",
#    "STRAND", "FLAGS", "VARIANT_CLASS", "SYMBOL_SOURCE", "HGNC_ID", "CANONICAL",
#    "MANE_SELECT", "MANE_PLUS_CLINICAL", "TSL", "APPRIS", "CCDS", "ENSP", "SWISSPROT",
#    "TREMBL", "UNIPARC", "UNIPROT_ISOFORM", "SOURCE", "GENE_PHENO", "SIFT", "PolyPhen",
#    "DOMAINS", "miRNA", "HGVS_OFFSET", "AF", "AFR_AF", "AMR_AF", "EAS_AF", "EUR_AF",
#    "SAS_AF", "gnomADe_AF", "gnomADe_AFR_AF", "gnomADe_AMR_AF", "gnomADe_ASJ_AF",
#    "gnomADe_EAS_AF", "gnomADe_FIN_AF", "gnomADe_NFE_AF", "gnomADe_OTH_AF", "gnomADe_SAS_AF",
#    "gnomADg_AF", "gnomADg_AFR_AF", "gnomADg_AMI_AF", "gnomADg_AMR_AF", "gnomADg_ASJ_AF",
#    "gnomADg_EAS_AF", "gnomADg_FIN_AF", "gnomADg_MID_AF", "gnomADg_NFE_AF", "gnomADg_OTH_AF",
#    "gnomADg_SAS_AF", "MAX_AF", "MAX_AF_POPS", "CLIN_SIG", "SOMATIC", "PHENO", "PUBMED",
#    "MOTIF_NAME", "MOTIF_POS", "HIGH_INF_POS", "MOTIF_SCORE_CHANGE", "TRANSCRIPTION_FACTORS",
#    "ClinVar", "ClinVar_CLNSIG", "ClinVar_CLNREVSTAT", "ClinVar_CLNDN"]

column_names = ["Allele", "Consequence", "IMPACT", "SYMBOL", "Gene", "Feature_type", "Feature", "BIOTYPE", "EXON", "INTRON", "HGVSc", "HGVSp", "cDNA_position", "CDS_position", "Protein_position", "Amino_acids", "Codons", "Existing_variation", "DISTANCE", "STRAND", "FLAGS", "SYMBOL_SOURCE", "HGNC_ID", "SOURCE", "ClinVar", "ClinVar_CLNSIG", "ClinVar_CLNREVSTAT", "ClinVar_CLNDN"]

# Initialize an empty DataFrame to store the extracted values
extracted_df = pd.DataFrame(columns=column_names)

# Helper function to remove leading and trailing commas
def remove_leading_trailing_commas(s):
    if isinstance(s, str):
        return s.strip(',')
    return s

# Iterate over each transcript and extract values into separate columns
for transcript in transcripts:
    values = {}
    for entry in transcript:
        entry = remove_leading_trailing_commas(entry)  # Remove leading and trailing commas
        parts = entry.split('|')
        for i, column_name in enumerate(column_names):
            if i < len(parts):
                if column_name in values:
                    values[column_name] += ',' + parts[i]
                else:
                    values[column_name] = parts[i]
    extracted_df = extracted_df.append(values, ignore_index=True)

# Apply set to each cell to get unique values
unique_extracted_df = extracted_df.applymap(lambda x: ','.join(set(x.split(',')) if isinstance(x, str) else ''))

# Remove leading and trailing commas in the entire DataFrame
unique_extracted_df = unique_extracted_df.applymap(remove_leading_trailing_commas)

# Display the DataFrame with unique values
unique_extracted_df

,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,SYMBOL_SOURCE,HGNC_ID,SOURCE,ClinVar,ClinVar_CLNSIG,ClinVar_CLNREVSTAT,ClinVar_CLNDN
0,G,missense_variant,MODERATE,OR4F5,ENSG00000186092,Transcript,ENST00000641515,protein_coding,3/3,,,,544,484,162,T/A,Aca/Gca,,,1,,HGNC,HGNC:14825,,,,,
1,C,synonymous_variant,LOW,OR4F5,ENSG00000186092,Transcript,ENST00000641515,protein_coding,3/3,,,,930,870,290,S,tcT/tcC,,,1,,HGNC,HGNC:14825,,,,,
2,A,3_prime_UTR_variant,MODIFIER,DPYD,ENSG00000188641,Transcript,ENST00000370192,protein_coding,23/23,,,,3970,,,,,,,-1,,HGNC,HGNC:3012,,100064,Benign,criteria_provided&_single_submitter,Dihydropyrimidine_dehydrogenase_deficiency&not...
3,A,3_prime_UTR_variant,MODIFIER,DPYD,ENSG00000188641,Transcript,ENST00000370192,protein_coding,23/23,,,,3776,,,,,,,-1,,HGNC,HGNC:3012,,298274,Uncertain_significance,criteria_provided&_single_submitter,Dihydropyrimidine_dehydrogenase_deficiency
4,T,3_prime_UTR_variant,MODIFIER,DPYD,ENSG00000188641,Transcript,ENST00000370192,protein_coding,23/23,,,,3763,,,,,,,-1,,HGNC,HGNC:3012,,100067,Benign,criteria_provided&_single_submitter,Dihydropyrimidine_dehydrogenase_deficiency&not...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,A,"downstream_gene_variant,intron_variant,intron_...",MODIFIER,CFTR,"ENSG00000001626,ENSG00000234001",Transcript,"ENST00000649406,ENST00000699598,ENST0000068727...","processed_transcript,processed_pseudogene,prot...",,"13/28,13/27,13/26,12/25,10/16,12/20,13/29",,,,,,,,,"4175,2842","-1,1",cds_end_NF,HGNC,HGNC:1884,,818094,Benign,criteria_provided&_multiple_submitters&_no_con...,Cystic_fibrosis&not_provided
101,G,"intron_variant,synonymous_variant&NMD_transcri...","LOW,MODIFIER",CFTR,ENSG00000001626,Transcript,"ENST00000649781,ENST00000699602,ENST0000068501...","protein_coding,nonsense_mediated_decay","15/29,15/30,15/28,14/26,14/21,15/27",10/16,,,"2472,2644,2669,2664,2558,2537,2632,2463","2472,2562,2379,2136","854,824,793,712",T,acT/acG,,,1,cds_end_NF,HGNC,HGNC:1884,,43577,Benign/Likely_benign,criteria_provided&_multiple_submitters&_no_con...,CFTR-related_disorders&Hereditary_pancreatitis...
102,A,"intron_variant&non_coding_transcript_variant,i...",MODIFIER,CFTR,"ENSG00000083622,ENSG00000001626",Transcript,"ENST00000649406,ENST00000699598,ENST0000068727...","nonsense_mediated_decay,protein_coding,lncRNA",,"2/2,21/26,20/25,22/28,21/28,1/6,20/20,21/27,21...",,,,,,,,,,"-1,1","cds_start_NF,cds_end_NF",HGNC,HGNC:1884,,818096,Benign,criteria_provided&_multiple_submitters&_no_con...,Cystic_fibrosis&not_provided
103,A,"non_coding_transcript_exon_variant,intron_vari...","LOW,MODIFIER",CFTR,ENSG00000001626,Transcript,"ENST00000649781,ENST00000699602,ENST0000068501...","protein_coding,nonsense_mediated_decay,retaine...","27/27,7/7,26/26,27/29,28/28,3/3,28/30","3/3,27/28",,,"4299,1231,4490,4566,4556,4296,3900,4459,4290,4...","4299,,4383,4206,3963,4389","1463,1461,1321,1402,1433",Q,caG/caA,,,1,"cds_start_NF,cds_end_NF",HGNC,HGNC:1884,,93157,Benign/Likely_benign,criteria_provided&_multiple_submitters&_no_con...,CFTR-related_disorders&Hereditary_pancreatitis...


In [ ]:
combined_df

In [39]:
combined_df.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/my_sample_data.xlsx', index=False)

In [23]:
# Extract the CSQ field from the INFO column
vcf['CSQ'] = vcf['INFO'].str.extract('CSQ=([^;]+)')

# Split the CSQ field into individual transcripts
transcripts = vcf['CSQ'].str.split(',')

# Define the column names
column_names = [
    "Allele", "Consequence", "IMPACT", "SYMBOL", "Gene", "Feature_type", "Feature",
    "BIOTYPE", "EXON", "INTRON", "HGVSc", "HGVSp", "cDNA_position", "CDS_position",
    "Protein_position", "Amino_acids", "Codons", "Existing_variation", "DISTANCE",
    "STRAND", "FLAGS", "VARIANT_CLASS", "SYMBOL_SOURCE", "HGNC_ID", "CANONICAL",
    "MANE_SELECT", "MANE_PLUS_CLINICAL", "TSL", "APPRIS", "CCDS", "ENSP", "SWISSPROT",
    "TREMBL", "UNIPARC", "UNIPROT_ISOFORM", "SOURCE", "GENE_PHENO", "SIFT", "PolyPhen",
    "DOMAINS", "miRNA", "HGVS_OFFSET", "AF", "AFR_AF", "AMR_AF", "EAS_AF", "EUR_AF",
    "SAS_AF", "gnomADe_AF", "gnomADe_AFR_AF", "gnomADe_AMR_AF", "gnomADe_ASJ_AF",
    "gnomADe_EAS_AF", "gnomADe_FIN_AF", "gnomADe_NFE_AF", "gnomADe_OTH_AF", "gnomADe_SAS_AF",
    "gnomADg_AF", "gnomADg_AFR_AF", "gnomADg_AMI_AF", "gnomADg_AMR_AF", "gnomADg_ASJ_AF",
    "gnomADg_EAS_AF", "gnomADg_FIN_AF", "gnomADg_MID_AF", "gnomADg_NFE_AF", "gnomADg_OTH_AF",
    "gnomADg_SAS_AF", "MAX_AF", "MAX_AF_POPS", "CLIN_SIG", "SOMATIC", "PHENO", "PUBMED",
    "MOTIF_NAME", "MOTIF_POS", "HIGH_INF_POS", "MOTIF_SCORE_CHANGE", "TRANSCRIPTION_FACTORS",
    "ClinVar", "ClinVar_CLNSIG", "ClinVar_CLNREVSTAT", "ClinVar_CLNDN"
]

# Initialize an empty DataFrame to store the extracted values
extracted_df = pd.DataFrame(columns=column_names)

# Iterate over each transcript and extract values into separate columns
for transcript in transcripts:
    values = {}
    for entry in transcript:
        parts = entry.split('|')
        for i, column_name in enumerate(column_names):
            if i < len(parts):
                if column_name in values:
                    values[column_name] += ',' + parts[i]
                else:
                    values[column_name] = parts[i]
    extracted_df = extracted_df.append(values, ignore_index=True)

# Display the extracted DataFrame with columns and values
extracted_df

,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,MANE_SELECT,MANE_PLUS_CLINICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,UNIPROT_ISOFORM,SOURCE,GENE_PHENO,SIFT,PolyPhen,DOMAINS,miRNA,HGVS_OFFSET,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,gnomADe_AF,gnomADe_AFR_AF,gnomADe_AMR_AF,gnomADe_ASJ_AF,gnomADe_EAS_AF,gnomADe_FIN_AF,gnomADe_NFE_AF,gnomADe_OTH_AF,gnomADe_SAS_AF,gnomADg_AF,gnomADg_AFR_AF,gnomADg_AMI_AF,gnomADg_AMR_AF,gnomADg_ASJ_AF,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF,MAX_AF,MAX_AF_POPS,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,ClinVar,ClinVar_CLNSIG,ClinVar_CLNREVSTAT,ClinVar_CLNDN
0,G,missense_variant,MODERATE,OR4F5,ENSG00000186092,Transcript,ENST00000641515,protein_coding,3/3,,,,544,484,162,T/A,Aca/Gca,,,1,,HGNC,HGNC:14825,,,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C,synonymous_variant,LOW,OR4F5,ENSG00000186092,Transcript,ENST00000641515,protein_coding,3/3,,,,930,870,290,S,tcT/tcC,,,1,,HGNC,HGNC:14825,,,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A,3_prime_UTR_variant,MODIFIER,DPYD,ENSG00000188641,Transcript,ENST00000370192,protein_coding,23/23,,,,3970,,,,,,,-1,,HGNC,HGNC:3012,,100064,Benign,criteria_provided&_single_submitter,Dihydropyrimidine_dehydrogenase_deficiency&not...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A,3_prime_UTR_variant,MODIFIER,DPYD,ENSG00000188641,Transcript,ENST00000370192,protein_coding,23/23,,,,3776,,,,,,,-1,,HGNC,HGNC:3012,,298274,Uncertain_significance,criteria_provided&_single_submitter,Dihydropyrimidine_dehydrogenase_deficiency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,T,3_prime_UTR_variant,MODIFIER,DPYD,ENSG00000188641,Transcript,ENST00000370192,protein_coding,23/23,,,,3763,,,,,,,-1,,HGNC,HGNC:3012,,100067,Benign,criteria_provided&_single_submitter,Dihydropyrimidine_dehydrogenase_deficiency&not...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,"A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A","intron_variant,intron_variant,upstream_gene_va...","MODIFIER,MODIFIER,MODIFIER,MODIFIER,MODIFIER,M...","CFTR,CFTR,,CFTR,CFTR,CFTR,CFTR,CFTR,CFTR,CFTR,...","ENSG00000001626,ENSG00000001626,ENSG0000023400...","Transcript,Transcript,Transcript,Transcript,Tr...","ENST00000003084,ENST00000426809,ENST0000044820...","protein_coding,protein_coding,processed_pseudo...",",,,,,,,,,,,,,,,,,,","13/26,12/25,,,13/26,13/26,10/16,12/20,12/25,13...",",,,,,,,,,,,,,,,,,,",",,,,,,,,,,,,,,,,,,",",,,,,,,,,,,,,,,,,,",",,,,,,,,,,,,,,,,,,",",,,,,,,,,,,,,,,,,,",",,,,,,,,,,,,,,,,,,",",,,,,,,,,,,,,,,,,,",",,,,,,,,,,,,,,,,,,

In [21]:
# Reset the index of both DataFrames
vcf.reset_index(drop=True, inplace=True)
extracted_df.reset_index(drop=True, inplace=True)
combined_df = pd.concat([vcf, extracted_df], axis=1)
combined_df

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE,Gene,Matched_gene,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RBQ,ABQ,RDF,RDR,ADF,ADR,ADP,WT,HET,HOM,NC,CDA,OTH,S3D,WTD,dbSNPBuildID,SLO,NSF,R3,R5,NSN,NSM,G5A,COMMON,RS,RV,TPA,CFL,GNO,VLD,ASP,ASS,Ref,U3,U5,TOPMED,WGT,MTP,LSD,NOC,DSS,SYN,KGPhase3,CAF,VC,MUT,KGPhase1,NOV,VP,SAO,GENEINFO,INT,G5,OM,PMC,SSR,RSPOS,HD,PM,ClinVar,ClinVar_CLNSIG,HET_new,Zygosity,CSQ,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,MANE_SELECT,MANE_PLUS_CLINICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,UNIPROT_ISOFORM,SOURCE,GENE_PHENO,SIFT,PolyPhen,DOMAINS,miRNA,HGVS_OFFSET,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,gnomADe_AF,gnomADe_AFR_AF,gnomADe_AMR_AF,gnomADe_ASJ_AF,gnomADe_EAS_AF,gnomADe_FIN_AF,gnomADe_NFE_AF,gnomADe_OTH_AF,gnomADe_SAS_AF,gnomADg_AF,gnomADg_AFR_AF,gnomADg_AMI_AF,gnomADg_AMR_AF,gnomADg_ASJ_AF,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF,MAX_AF,MAX_AF_POPS,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,ClinVar,ClinVar_CLNSIG,ClinVar_CLNREVSTAT,ClinVar_CLNDN
0,chr1,69511,rs2691305,A,G,.,PASS,ADP=90;WT=0;HET=0;HOM=1;NC=0;ASP;G5;GENEINFO=O...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:255:90:90:0:90:100%:1.0988E-53:0:54:0:0:71:19,OR4F5,Yes,1/1,255,90,90,0,90,100%,1.0988E-53,0,54,0,0,71,19,ADP=90,WT=0,HET=0,HOM=1,NC=0,,,,,dbSNPBuildID=100,,,,,,,,,RS=2691305,,,,,,,,,,,"TOPMED=0.12468941131498470,0.87531058868501529,.",WGT=1,,,,,,,,VC=SNV,,,,VP=0x050100000a05050116000100,SAO=0,GENEINFO=OR4F5:79501,,,,,SSR=0,RSPOS=69511,,,,,0,Homozygous,G|missense_variant|MODERATE|OR4F5|ENSG00000186...,G,missense_variant,MODERATE,OR4F5,ENSG00000186092,Transcript,ENST00000641515,protein_coding,3/3,,,,544,484,162,T/A,Aca/Gca,,,1,,HGNC,HGNC:14825,,,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,69897,rs200676709,T,C,.,PASS,"ADP=96;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.3119,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:72:96:96:74:22:22.92%:6.1288E-8:54:53:51:2...,OR4F5,Yes,0/1,72,96,96,74,22,22.92%,6.1288E-8,54,53,51,23,15,7,ADP=96,WT=0,HET=1,HOM=0,NC=0,,,,,dbSNPBuildID=137,,,,,,,,COMMON=1,RS=200676709,,,,,,,,,,,"TOPMED=0.71955434505606523,0.28044565494393476",WGT=1,,,,,,,"CAF=0.3119,0.6881",VC=SNV,,,,VP=0x050300000305170026000100,SAO=0,GENEINFO=OR4F5:79501,,,,,SSR=0,RSPOS=69897,,,,,1,Heterozygous,C|synonymous_variant|LOW|OR4F5|ENSG00000186092...,C,synonymous_variant,LOW,OR4F5,ENSG00000186092,Transcript,ENST00000641515,protein_coding,3/3,,,,930,870,290,S,tcT/tcC,,,1,,HGNC,HGNC:14825,,,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,97078196,rs291593,G,A,.,PASS,"ADP=8;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.615,0.38...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:41:8:8:0:8:100%:7.77E-5:0:52:0:0:8:0,DPYD,Yes,1/1,41,8,8,0,8,100%,7.77E-5,0,52,0,0,8,0,ADP=8,WT=0,HET=0,HOM=1,NC=0,,,,,dbSNPBuildID=79,,,,,,,,COMMON=1,RS=291593,,,,,,,,,,,"TOPMED=0.70939252038735983,0.29060747961264016",WGT=1,,,,,,,"CAF=0.615,0.385",VC=SNV,,,,VP=0x05016880000517053e100100,SAO=1,GENEINFO=DPYD:1806,,,,,SSR=0,RSPOS=97078196,,,,,0,Homozygous,A|3_prime_UTR_variant|MODIFIER|DPYD|ENSG000001...,A,3_prime_UTR_variant,MODIFIER,DPYD,ENSG00000188641,Transcript,ENST00000370192,protein_coding,23/23,,,,3970,,,,,,,-1,,HGNC,HGNC:3012,,100064,Benign,criteria_provided&_single_submitter,Dihydropyrimidine_dehydrogenase_deficiency&not...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [25]:
import pandas as pd

# Sample DataFrame with an INFO column
data = {'INFO': ['ADP=64;WT=0;HET=1;HOM=0;NC=0;ASP;G5;G5A;GENEINFO=OR4F5:79501;REF;RS=201219564;RSPOS=69270;S3D;SAO=0;SLO;SSR=0;SYN;TOPMED=0.72539181957186544,0.27459225280326197,0.00001592762487257;VC=SNV;VLD;VP=0x050300000305070002000100;WGT=1;dbSNPBuildID=137;CSQ=G|synonymous_variant|LOW|OR4F5|ENSG00000186092|Transcript|ENST00000641515|protein_coding|3/3||ENST00000641515.2:c.243A>G|ENSP00000493376.2:p.Ser81%3D|303|243|81|S|tcA/tcG|rs201219564&COSV58736820||1||SNV|HGNC|HGNC:14825|YES|NM_001005484.2|||P1||ENSP00000493376||A0A2U3U0J3.17|UPI000D1938F0||||||Low_complexity_(Seg):seg&PROSITE_profiles:PS50262&CDD:cd15226&PANTHER:PTHR48002&PANTHER:PTHR48002:SF51&Gene3D:1.20.1070.10&Pfam:PF13853&Superfamily:SSF81321&Prints:PR00237|||||||||0.838|0.3591|0.7932|0.8482|0.9984|0.8821|0.9146|0.868|0.9005|0.6291|0.3544|0.8158|0.6945|0.8033|0.9961|0.9294|0.7857|0.8798|0.6087|0.8855|0.9984|gnomADe_EAS||0&1|0&1||||||||||,G|regulatory_region_variant|MODIFIER|||RegulatoryFeature|ENSR00000918279|TF_binding_site||||||||||rs201219564&COSV58736820||||SNV|||||||||||||||||||||||||||0.838|0.3591|0.7932|0.8482|0.9984|0.8821|0.9146|0.868|0.9005|0.6291|0.3544|0.8158|0.6945|0.8033|0.9961|0.9294|0.7857|0.8798|0.6087|0.8855|0.9984|gnomADe_EAS||0&1|0&1||||||||||']}
df = pd.DataFrame(data)

# Extract the CSQ field from the INFO column
df['CSQ'] = df['INFO'].str.extract('CSQ=([^;]+)')

# Split the CSQ field into individual transcripts
transcripts = df['CSQ'].str.split(',')

# Define the column names
column_names = [
    "Allele", "Consequence", "IMPACT", "SYMBOL", "Gene", "Feature_type", "Feature",
    "BIOTYPE", "EXON", "INTRON", "HGVSc", "HGVSp", "cDNA_position", "CDS_position",
    "Protein_position", "Amino_acids", "Codons", "Existing_variation", "DISTANCE",
    "STRAND", "FLAGS", "VARIANT_CLASS", "SYMBOL_SOURCE", "HGNC_ID", "CANONICAL",
    "MANE_SELECT", "MANE_PLUS_CLINICAL", "TSL", "APPRIS", "CCDS", "ENSP", "SWISSPROT",
    "TREMBL", "UNIPARC", "UNIPROT_ISOFORM", "SOURCE", "GENE_PHENO", "SIFT", "PolyPhen",
    "DOMAINS", "miRNA", "HGVS_OFFSET", "AF", "AFR_AF", "AMR_AF", "EAS_AF", "EUR_AF",
    "SAS_AF", "gnomADe_AF", "gnomADe_AFR_AF", "gnomADe_AMR_AF", "gnomADe_ASJ_AF",
    "gnomADe_EAS_AF", "gnomADe_FIN_AF", "gnomADe_NFE_AF", "gnomADe_OTH_AF", "gnomADe_SAS_AF",
    "gnomADg_AF", "gnomADg_AFR_AF", "gnomADg_AMI_AF", "gnomADg_AMR_AF", "gnomADg_ASJ_AF",
    "gnomADg_EAS_AF", "gnomADg_FIN_AF", "gnomADg_MID_AF", "gnomADg_NFE_AF", "gnomADg_OTH_AF",
    "gnomADg_SAS_AF", "MAX_AF", "MAX_AF_POPS", "CLIN_SIG", "SOMATIC", "PHENO", "PUBMED",
    "MOTIF_NAME", "MOTIF_POS", "HIGH_INF_POS", "MOTIF_SCORE_CHANGE", "TRANSCRIPTION_FACTORS",
    "ClinVar", "ClinVar_CLNSIG", "ClinVar_CLNREVSTAT", "ClinVar_CLNDN"
]

# Initialize an empty DataFrame to store the extracted values
extracted_df = pd.DataFrame(columns=column_names)

# Iterate over each transcript and extract values into separate columns
for transcript in transcripts:
    values = {}
    for entry in transcript:
        parts = entry.split('|')
        for i, column_name in enumerate(column_names):
            if i < len(parts):
                if column_name in values:
                    values[column_name] += ',' + parts[i]
                else:
                    values[column_name] = parts[i]
    extracted_df = extracted_df.append(values, ignore_index=True)

# Apply set to each cell to get unique values
unique_extracted_df = extracted_df.applymap(lambda x: ','.join(set(x.split(',')) if x else ''))

# Display the DataFrame with unique values
unique_extracted_df

,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,MANE_SELECT,MANE_PLUS_CLINICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,UNIPROT_ISOFORM,SOURCE,GENE_PHENO,SIFT,PolyPhen,DOMAINS,miRNA,HGVS_OFFSET,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,gnomADe_AF,gnomADe_AFR_AF,gnomADe_AMR_AF,gnomADe_ASJ_AF,gnomADe_EAS_AF,gnomADe_FIN_AF,gnomADe_NFE_AF,gnomADe_OTH_AF,gnomADe_SAS_AF,gnomADg_AF,gnomADg_AFR_AF,gnomADg_AMI_AF,gnomADg_AMR_AF,gnomADg_ASJ_AF,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF,MAX_AF,MAX_AF_POPS,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,ClinVar,ClinVar_CLNSIG,ClinVar_CLNREVSTAT,ClinVar_CLNDN
0,G,"regulatory_region_variant,synonymous_variant","LOW,MODIFIER",",OR4F5",",ENSG00000186092","RegulatoryFeature,Transcript","ENSR00000918279,ENST00000641515","protein_coding,TF_binding_site",",3/3",,",ENST00000641515.2:c.243A>G",",ENSP00000493376.2:p.Ser81%3D",",303",",243","81,",",S","tcA/tcG,",rs201219564&COSV58736820,,",1",,SNV,",HGNC",",HGNC:14825",",YES",",NM_001005484.2",,,",P1",,",ENSP00000493376",,"A0A2U3U0J3.17,",",UPI000D1938F0",,,,,,",Low_complexity_(Seg):seg&PROSITE_profiles:PS5...",,,,,,,,,0.838,0.3591,0.7932,0.8482,0.9984,0.8821,0.9146,0.868,0.9005,0.6291,0.3544,0.8158,0.6945,0.8033,0.9961,0.9294,0.7857,0.8798,0.6087,0.8855,0.9984,gnomADe_EAS,,0&1,0&1,,,,,,,,,,


In [5]:
df_gene = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/selective_genes.xlsx')
df_gene

,Gene
0,ABCG2
1,CACNA1S
2,CFTR
3,CYP2B6
4,CYP2C19
5,CYP2C9
6,CYP2D6
7,CYP3A5
8,DPYD
9,G6PD


In [7]:
vcf['Matched_gene'] = 'No'

# Iterate through each gene in df1
for genes in vcf['Gene']:
    if isinstance(genes, str):  # Check if the gene value is a non-null string
        gene_list = genes.split(',')  # Split the genes by comma to create a list
        match = any(gene in df_gene['Gene'].values for gene in gene_list)  # Check if any gene in the list exists in df2
        if match:
            vcf.loc[vcf['Gene'] == genes, 'Matched_gene'] = 'Yes'
            
vcf = vcf[vcf['Matched_gene'] == 'Yes']
vcf

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE,Gene,Matched_gene
6,chr1,69511,rs2691305,A,G,.,PASS,ADP=90;WT=0;HET=0;HOM=1;NC=0;ASP;G5;GENEINFO=O...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:255:90:90:0:90:100%:1.0988E-53:0:54:0:0:71:19,OR4F5,Yes
7,chr1,69897,rs200676709,T,C,.,PASS,"ADP=96;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.3119,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:72:96:96:74:22:22.92%:6.1288E-8:54:53:51:2...,OR4F5,Yes
2548,chr1,97078196,rs291593,G,A,.,PASS,"ADP=8;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.615,0.38...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:41:8:8:0:8:100%:7.77E-5:0:52:0:0:8:0,DPYD,Yes
2549,chr1,97078390,rs886046571,G,A,.,PASS,ADP=12;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DPYD...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:31:12:12:4:8:66.67%:6.7304E-4:34:60:2:2:7:1,DPYD,Yes
2550,chr1,97078403,rs1042482,C,T,.,PASS,"ADP=8;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.9105,0.0...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:41:8:8:0:8:100%:7.77E-5:0:48:0:0:6:2,DPYD,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...
60871,chr7,117590591,rs4148711,T,A,.,PASS,"ADP=15;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.5164,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:81:15:15:0:15:100%:6.4467E-9:0:51:0:0:6:9,CFTR,Yes
60872,chr7,117595001,rs1042077,T,G,.,PASS,"ADP=43;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.5208,.,...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:248:43:43:0:43:100%:1.5066E-25:0:57:0:0:32:11,CFTR,Yes
60873,chr7,117627457,rs213989,C,A,.,PASS,"ADP=29;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.856,0.1...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:164:29:29:0:29:100%:3.3259E-17:0:56:0:0:23:6,CFTR,Yes
60874,chr7,117667054,rs1800136,G,A,.,PASS,"ADP=17;WT=0;HET=0;HOM=1;NC=0;ASP;CAF=0.8139,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,1/1:93:17:17:0:17:100%:4.2852E-10:0:59:0:0:13:4,CFTR,Yes


In [1]:
import pandas as pd

# Read the VCF file into a Pandas DataFrame (skip first 80 rows as shown in a previous answer)
vcf = pd.read_csv(
    r'C:/Users/GenepoweRx_Madhu/Downloads/718_final.vcf',
    comment='#',
    sep='\t',
    header=None,
    low_memory=False,
    skiprows=80
)

# Assign appropriate column names
vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']

sample_cols = vcf['SAMPLE'].str.split(':', expand=True)
sample_cols.columns = ['GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RBQ', 'ABQ', 'RDF', 'RDR', 'ADF', 'ADR']

# Assign the values to the newly created columns
vcf = pd.concat([vcf, sample_cols], axis=1)

vcf["Gene"] = vcf["INFO"].str.extract('GENEINFO=(?P<GENEINFO>.+?);')
vcf['Gene'] = vcf['Gene'].apply(lambda x: ','.join(set([segment.split(':')[0] for segment in x.split('|')])) if pd.notnull(x) else '')

# Create empty columns
columns = ['ADP', 'WT', 'HET', 'HOM', 'NC', 'CDA', 'OTH', 'S3D', 'WTD', 'dbSNPBuildID', 'SLO',
           'NSF', 'R3', 'R5', 'NSN', 'NSM', 'G5A', 'COMMON', 'RS', 'RV', 'TPA', 'CFL', 'GNO',
           'VLD', 'ASP', 'ASS', 'REF', 'U3', 'U5', 'TOPMED', 'WGT', 'MTP', 'LSD', 'NOC',
           'DSS', 'SYN', 'KGPhase3', 'CAF', 'VC', 'MUT', 'KGPhase1', 'NOV', 'VP', 'SAO',
           'GENEINFO', 'INT', 'G5', 'OM', 'PMC', 'SSR', 'RSPOS', 'HD', 'PM', 'ClinVar',
           'ClinVar_CLNSIG']

for col in columns:
    vcf[col] = ''

# Populate columns based on 'INFO' values
for i, row in vcf.iterrows():
    info = row['INFO']
    items = info.split(';')
    for item in items:
        key_value = item.split('=')
        key = key_value[0]
        if key in columns:
            if len(key_value) > 1:
                value = key_value[1]
                vcf.at[i, key] = f"{key}={value}"
            else:
                vcf.at[i, key] = key

# Extract the numeric part of the HET column and convert it to an integer
vcf['HET_new'] = vcf['HET'].str.extract('(\d+)').astype(int)

# Create the Zygosity column based on the HET column
vcf['Zygosity'] = vcf['HET_new'].apply(lambda x: 'Heterozygous' if x == 1 else 'Homozygous')
vcf

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE,...,OM,PMC,SSR,RSPOS,HD,PM,ClinVar,ClinVar_CLNSIG,HET_new,Zygosity
0,chr1,14930,rs6682385,,G,.,PASS,"ADP=51;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5178,0....",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:92:51:51:26:25:49.02%:5.8079E-10:41:51:10:...,...,,,SSR=0,RSPOS=14930,,,,,1,Heterozygous
1,chr1,15118,rs11580262,,G,.,PASS,ADP=20;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=WASH...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:23:20:20:13:7:35%:4.158E-3:42:45:8:5:4:3,...,,,SSR=0,RSPOS=15118,,,,,1,Heterozygous
2,chr1,16495,rs3210724,,C,.,PASS,ADP=34;WT=0;HET=1;HOM=0;NC=0;ASP;G5;G5A;GENEIN...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:107:34:34:9:25:73.53%:1.9819E-11:37:44:8:1...,...,,,SSR=0,RSPOS=16495,,,,,1,Heterozygous
3,chr1,16949,rs199745162,,C,.,PASS,"ADP=85;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.986,0.0...",GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:55:85:85:68:17:20%:3.1403E-6:52:39:55:13:11:6,...,,,SSR=0,RSPOS=16949,,,,,1,Heterozygous
4,chr1,24786,rs11266852,,A,.,PASS,ADP=93;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=WASH...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:190:93:93:43:50:53.76%:9.1368E-20:63:55:39...,...,,,SSR=0,RSPOS=24786,,,,,1,Heterozygous
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71451,chrY,57209899,.,,T,.,PASS,ADP=21;WT=0;HET=1;HOM=0;NC=0;CSQ=T|non_coding_...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:40:21:21:10:11:52.38%:8.2399E-5:62:54:10:0...,...,,,,,,,,,1,Heterozygous
71452,chrY,57210016,.,,T,.,PASS,ADP=16;WT=0;HET=1;HOM=0;NC=0;CSQ=T|intron_vari...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:33:16:16:7:9:56.25%:4.0786E-4:74:64:7:0:8:1,...,,,,,,,,,1,Heterozygous
71453,chrY,57210723,.,,A,.,PASS,ADP=37;WT=0;HET=1;HOM=0;NC=0;CSQ=A|non_coding_...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:29:37:37:28:9:24.32%:1.1256E-3:68:68:28:0:9:0,...,,,,,,,,,1,Heterozygous
71454,chrY,57210771,.,,C,.,PASS,ADP=30;WT=0;HET=1;HOM=0;NC=0;CSQ=C|non_coding_...,GT:GQ:SDP:DP:RD:AD:FREQ:PVAL:RBQ:ABQ:RDF:RDR:A...,0/1:30:30:30:21:9:30%:9.678E-4:59:65:17:4:8:1,...,,,,,,,,,1,Heterozygous


In [19]:
import pandas as pd

# Sample DataFrame with an INFO column
data = {'INFO': ['ADP=64;WT=0;HET=1;HOM=0;NC=0;ASP;G5;G5A;GENEINFO=OR4F5:79501;REF;RS=201219564;RSPOS=69270;S3D;SAO=0;SLO;SSR=0;SYN;TOPMED=0.72539181957186544,0.27459225280326197,0.00001592762487257;VC=SNV;VLD;VP=0x050300000305070002000100;WGT=1;dbSNPBuildID=137;CSQ=G|synonymous_variant|LOW|OR4F5|ENSG00000186092|Transcript|ENST00000641515|protein_coding|3/3||ENST00000641515.2:c.243A>G|ENSP00000493376.2:p.Ser81%3D|303|243|81|S|tcA/tcG|rs201219564&COSV58736820||1||SNV|HGNC|HGNC:14825|YES|NM_001005484.2|||P1||ENSP00000493376||A0A2U3U0J3.17|UPI000D1938F0||||||Low_complexity_(Seg):seg&PROSITE_profiles:PS50262&CDD:cd15226&PANTHER:PTHR48002&PANTHER:PTHR48002:SF51&Gene3D:1.20.1070.10&Pfam:PF13853&Superfamily:SSF81321&Prints:PR00237|||||||||0.838|0.3591|0.7932|0.8482|0.9984|0.8821|0.9146|0.868|0.9005|0.6291|0.3544|0.8158|0.6945|0.8033|0.9961|0.9294|0.7857|0.8798|0.6087|0.8855|0.9984|gnomADe_EAS||0&1|0&1||||||||||,G|regulatory_region_variant|MODIFIER|||RegulatoryFeature|ENSR00000918279|TF_binding_site||||||||||rs201219564&COSV58736820||||SNV|||||||||||||||||||||||||||0.838|0.3591|0.7932|0.8482|0.9984|0.8821|0.9146|0.868|0.9005|0.6291|0.3544|0.8158|0.6945|0.8033|0.9961|0.9294|0.7857|0.8798|0.6087|0.8855|0.9984|gnomADe_EAS||0&1|0&1||||||||||']}
df = pd.DataFrame(data)

# Extract the CSQ field from the INFO column
df['CSQ'] = df['INFO'].str.extract('CSQ=([^;]+)')

# Split the CSQ field into individual transcripts
transcripts = df['CSQ'].str.split(',')

# Define the column names
column_names = [
    "Allele", "Consequence", "IMPACT", "SYMBOL", "Gene", "Feature_type", "Feature",
    "BIOTYPE", "EXON", "INTRON", "HGVSc", "HGVSp", "cDNA_position", "CDS_position",
    "Protein_position", "Amino_acids", "Codons", "Existing_variation", "DISTANCE",
    "STRAND", "FLAGS", "VARIANT_CLASS", "SYMBOL_SOURCE", "HGNC_ID", "CANONICAL",
    "MANE_SELECT", "MANE_PLUS_CLINICAL", "TSL", "APPRIS", "CCDS", "ENSP", "SWISSPROT",
    "TREMBL", "UNIPARC", "UNIPROT_ISOFORM", "SOURCE", "GENE_PHENO", "SIFT", "PolyPhen",
    "DOMAINS", "miRNA", "HGVS_OFFSET", "AF", "AFR_AF", "AMR_AF", "EAS_AF", "EUR_AF",
    "SAS_AF", "gnomADe_AF", "gnomADe_AFR_AF", "gnomADe_AMR_AF", "gnomADe_ASJ_AF",
    "gnomADe_EAS_AF", "gnomADe_FIN_AF", "gnomADe_NFE_AF", "gnomADe_OTH_AF", "gnomADe_SAS_AF",
    "gnomADg_AF", "gnomADg_AFR_AF", "gnomADg_AMI_AF", "gnomADg_AMR_AF", "gnomADg_ASJ_AF",
    "gnomADg_EAS_AF", "gnomADg_FIN_AF", "gnomADg_MID_AF", "gnomADg_NFE_AF", "gnomADg_OTH_AF",
    "gnomADg_SAS_AF", "MAX_AF", "MAX_AF_POPS", "CLIN_SIG", "SOMATIC", "PHENO", "PUBMED",
    "MOTIF_NAME", "MOTIF_POS", "HIGH_INF_POS", "MOTIF_SCORE_CHANGE", "TRANSCRIPTION_FACTORS",
    "ClinVar", "ClinVar_CLNSIG", "ClinVar_CLNREVSTAT", "ClinVar_CLNDN"
]

# Initialize an empty DataFrame to store the extracted values
extracted_df = pd.DataFrame(columns=column_names)

# Iterate over each transcript and extract values into separate columns
for transcript in transcripts:
    values = {}
    for entry in transcript:
        parts = entry.split('|')
        for i, column_name in enumerate(column_names):
            if i < len(parts):
                if column_name in values:
                    values[column_name] += ',' + parts[i]
                else:
                    values[column_name] = parts[i]
    extracted_df = extracted_df.append(values, ignore_index=True)

# Display the extracted DataFrame with columns and values
extracted_df

,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,MANE_SELECT,MANE_PLUS_CLINICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,UNIPROT_ISOFORM,SOURCE,GENE_PHENO,SIFT,PolyPhen,DOMAINS,miRNA,HGVS_OFFSET,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,gnomADe_AF,gnomADe_AFR_AF,gnomADe_AMR_AF,gnomADe_ASJ_AF,gnomADe_EAS_AF,gnomADe_FIN_AF,gnomADe_NFE_AF,gnomADe_OTH_AF,gnomADe_SAS_AF,gnomADg_AF,gnomADg_AFR_AF,gnomADg_AMI_AF,gnomADg_AMR_AF,gnomADg_ASJ_AF,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF,MAX_AF,MAX_AF_POPS,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,ClinVar,ClinVar_CLNSIG,ClinVar_CLNREVSTAT,ClinVar_CLNDN
0,"G,G","synonymous_variant,regulatory_region_variant","LOW,MODIFIER","OR4F5,","ENSG00000186092,","Transcript,RegulatoryFeature","ENST00000641515,ENSR00000918279","protein_coding,TF_binding_site","3/3,",",","ENST00000641515.2:c.243A>G,","ENSP00000493376.2:p.Ser81%3D,","303,","243,","81,","S,","tcA/tcG,","rs201219564&COSV58736820,rs201219564&COSV58736820",",","1,",",","SNV,SNV","HGNC,","HGNC:14825,","YES,","NM_001005484.2,",",",",","P1,",",","ENSP00000493376,",",","A0A2U3U0J3.17,","UPI000D1938F0,",",",",",",",",",",",Low_complexity_(Seg):seg&PROSITE_profiles:PS50...,",",",",",",",",",",",",",",",","0.838,0.838","0.3591,0.3591","0.7932,0.7932","0.8482,0.8482","0.9984,0.9984","0.8821,0.8821","0.9146,0.9146","0.868,0.868","0.9005,0.9005","0.6291,0.6291","0.3544,0.3544","0.8158,0.8158","0.6945,0.6945","0.8033,0.8033","0.9961,0.9961","0.9294,0.9294","0.7857,0.7857","0.8798,0.8798","0.6087,0.6087","0.8855,0.8855","0.9984,0.9984","gnomADe_EAS,gnomADe_EAS",",","0&1,0&1","0&1,0&1",",",",",",",",",",",",",",",",",",",","


In [ ]:
# Extract the CSQ field from the INFO column
vcf['CSQ'] = vcf['INFO'].str.extract('CSQ=([^;]+)')

# Split the CSQ field into individual transcripts
transcripts = vcf['CSQ'].str.split(',')

# Define the column names
column_names = [
    "Allele", "Consequence", "IMPACT", "SYMBOL", "Gene", "Feature_type", "Feature",
    "BIOTYPE", "EXON", "INTRON", "HGVSc", "HGVSp", "cDNA_position", "CDS_position",
    "Protein_position", "Amino_acids", "Codons", "Existing_variation", "DISTANCE",
    "STRAND", "FLAGS", "VARIANT_CLASS", "SYMBOL_SOURCE", "HGNC_ID", "CANONICAL",
    "MANE_SELECT", "MANE_PLUS_CLINICAL", "TSL", "APPRIS", "CCDS", "ENSP", "SWISSPROT",
    "TREMBL", "UNIPARC", "UNIPROT_ISOFORM", "SOURCE", "GENE_PHENO", "SIFT", "PolyPhen",
    "DOMAINS", "miRNA", "HGVS_OFFSET", "AF", "AFR_AF", "AMR_AF", "EAS_AF", "EUR_AF",
    "SAS_AF", "gnomADe_AF", "gnomADe_AFR_AF", "gnomADe_AMR_AF", "gnomADe_ASJ_AF",
    "gnomADe_EAS_AF", "gnomADe_FIN_AF", "gnomADe_NFE_AF", "gnomADe_OTH_AF", "gnomADe_SAS_AF",
    "gnomADg_AF", "gnomADg_AFR_AF", "gnomADg_AMI_AF", "gnomADg_AMR_AF", "gnomADg_ASJ_AF",
    "gnomADg_EAS_AF", "gnomADg_FIN_AF", "gnomADg_MID_AF", "gnomADg_NFE_AF", "gnomADg_OTH_AF",
    "gnomADg_SAS_AF", "MAX_AF", "MAX_AF_POPS", "CLIN_SIG", "SOMATIC", "PHENO", "PUBMED",
    "MOTIF_NAME", "MOTIF_POS", "HIGH_INF_POS", "MOTIF_SCORE_CHANGE", "TRANSCRIPTION_FACTORS",
    "ClinVar", "ClinVar_CLNSIG", "ClinVar_CLNREVSTAT", "ClinVar_CLNDN"
]

# Initialize an empty DataFrame to store the extracted values
extracted_df = pd.DataFrame(columns=column_names)

# Iterate over each transcript and extract values into separate columns
for transcript in transcripts:
    values = {}
    for entry in transcript:
        parts = entry.split('|')
        for i, column_name in enumerate(column_names):
            if i < len(parts):
                if column_name in values:
                    values[column_name] += ',' + parts[i]
                else:
                    values[column_name] = parts[i]
    extracted_df = extracted_df.append(values, ignore_index=True)

# Display the extracted DataFrame with columns and values
extracted_df